### Librairies

In [1]:
def warn(*args, **kwargs):
    pass
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.warn = warn
import os
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.impute import SimpleImputer
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.under_sampling import RandomUnderSampler

from utility.utility import false_neg_scorer, false_pos_scorer
from sklearn.metrics import fbeta_score, make_scorer

from get_outcomes import get_mortality_6m
import warnings
#import shap

%matplotlib inline

In [6]:
DATA_DIRECTORY = "data/"

In [7]:
y = get_mortality_6m()

    name  mortality
0  P0000          0
1  P0001          0
2  P0002          0
3  P0003          0
4  P0004          0
Outcome events : 33


### Preprocessing features

In [8]:
X_volumes_clinical = pd.read_csv(DATA_DIRECTORY+"segmentation_data_anonymized.csv", usecols=range(1,31))
X_volumes_clinical.head()

,name,supratentorial_IPH,supratentorial_SAH,supratentorial_Petechiae,supratentorial_Edema,infratentorial_IPH,infratentorial_SAH,infratentorial_Petechiae,infratentorial_Edema,brainstem_IPH,...,pression_arterielle_diastolique_PAD_arrivee_du_smur,score_glasgow_initial,score_glasgow_moteur_initial,anomalie_pupillaire_prehospitalier,frequence_cardiaque_FC_arrivee_du_smur,arret_cardio_respiratoire_massage,penetrant_objet,ischemie_du_membre,hemorragie_externe,amputation
0,P0001,0,342,0,0,0,15,0,0,0,...,49.0,15.0,6.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0
1,P0002,0,0,0,0,0,0,0,0,0,...,60.0,15.0,6.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
2,P0003,0,101,0,0,0,0,0,0,0,...,64.0,14.0,6.0,0.0,120.0,0.0,0.0,0.0,0.0,0.0
3,P0004,0,328,0,0,0,0,0,0,0,...,71.0,15.0,6.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0
4,P0005,0,9,0,15,0,0,0,0,0,...,79.0,NaN,NaN,0.0,83.0,0.0,0.0,0.0,0.0,0.0


In [9]:
columns_to_load = list(range(16, 31)) + [1]

X_clinical_only = pd.read_csv(DATA_DIRECTORY+"segmentation_data_anonymized.csv", usecols=columns_to_load)
X_clinical_only.head()

,name,age,hemocue_initial,fracas_du_bassin,catecholamines,pression_arterielle_systolique_PAS_arrivee_du_smur,pression_arterielle_diastolique_PAD_arrivee_du_smur,score_glasgow_initial,score_glasgow_moteur_initial,anomalie_pupillaire_prehospitalier,frequence_cardiaque_FC_arrivee_du_smur,arret_cardio_respiratoire_massage,penetrant_objet,ischemie_du_membre,hemorragie_externe,amputation
0,P0001,52.0,NaN,0.0,0.0,87.0,49.0,15.0,6.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0
1,P0002,23.0,NaN,0.0,0.0,100.0,60.0,15.0,6.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
2,P0003,42.0,13.1,0.0,0.0,101.0,64.0,14.0,6.0,0.0,120.0,0.0,0.0,0.0,0.0,0.0
3,P0004,34.0,15.8,0.0,0.0,110.0,71.0,15.0,6.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0
4,P0005,22.0,NaN,0.0,0.0,114.0,79.0,NaN,NaN,0.0,83.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Identify indexes where 'mortality' is NaN in y
nan_indexes = y.loc[pd.isna(y["mortality"]), :].index
print(f"Indexes with NaN in 'mortality': {nan_indexes}")

# Drop rows with NaN in y
y = y.drop(index=nan_indexes)

# Ensure 'name' is the index or used for alignment
X_volumes_clinical = X_volumes_clinical[~X_volumes_clinical['name'].isin(y.loc[nan_indexes, 'name'])]

# Verify the shapes after cleaning
print(f"Shape of y after cleaning: {y.shape}")
print(f"Shape of X_volumes_clinical after cleaning: {X_volumes_clinical.shape}")

In [ ]:
# Vérifiez les valeurs uniques de 'name' dans y
print("Unique 'name' values in y:", y['name'].nunique())

# Vérifiez les valeurs uniques de 'name' dans X_volumes_clinical_with_outcome
print("Unique 'name' values in X_volumes_clinical:", X_volumes_clinical['name'].nunique())

In [ ]:
# Noms dans y mais pas dans X_volumes_clinical
missing_in_X = set(y['name']) - set(X_volumes_clinical['name'])
print(f"Names in y but not in X_volumes_clinical: {missing_in_X}")

# Noms dans X_volumes_clinical mais pas dans y
missing_in_y = set(X_volumes_clinical['name']) - set(y['name'])
print(f"Names in X_volumes_clinical but not in y: {missing_in_y}")

In [ ]:
# Trouver les noms communs
common_names = set(y['name']).intersection(set(X_volumes_clinical['name']))

# Filtrer y et X_volumes_clinical_with_outcome pour ne garder que les noms communs
y = y[y['name'].isin(common_names)]
X_volumes_clinical = X_volumes_clinical[X_volumes_clinical['name'].isin(common_names)]

# Vérifiez les dimensions après nettoyage
print(f"Shape of y after alignment: {y.shape}")
print(f"Shape of X_volumes_clinical_with_outcome after alignment: {X_volumes_clinical.shape}")

In [ ]:
# Check for duplicates in 'name' in y
print("Number of duplicate names in y:", y['name'].duplicated().sum())

# Check for duplicates in 'name' in X_volumes_clinical_with_outcome
print("Number of duplicate names in X_volumes_clinical:", X_volumes_clinical['name'].duplicated().sum())

In [ ]:
# Remove duplicates from both DataFrames
y = y.drop_duplicates(subset=['name'])
X_volumes_clinical = X_volumes_clinical.drop_duplicates(subset=['name'])

# Verify the shapes again
print(f"Shape of y after removing duplicates: {y.shape}")
print(f"Shape of X_volumes_clinical after removing duplicates: {X_volumes_clinical.shape}")

In [ ]:
X_volumes_clinical.head()

In [ ]:
# Check if 'name' values are the same in both DataFrames
common_names = set(y['name']) == set(X_volumes_clinical['name'])
print(f"Are 'name' values aligned between y and X_volumes_clinical? {common_names}")

In [ ]:
# Trier y et X par la colonne 'name'
y = y.sort_values(by='name').reset_index(drop=True)
X_volumes_clinical = X_volumes_clinical.sort_values(by='name').reset_index(drop=True)

# Vérifier si les noms sont alignés
print((y['name'].values == X_volumes_clinical['name'].values).all())

In [ ]:
# Drop the 'name' column from X_volumes_clinical_with_outcome
X_features = X_volumes_clinical.drop(columns=['name'])

# imputation with median 
X_features_imputed = X_features.fillna(X_features.median())

In [ ]:
# Drop the 'name' column from y (if it exists)
y_outcome = y.drop(columns=['name'])

# Ensure y_outcome is a 1D array
y_outcome = y_outcome.values.ravel()  # Convert to 1D if using pandas DataFrame

### Gradient boosting with traumatrix model + segmentation

In [ ]:
ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [ ]:
FOLDS = 5
N_REPEATS = 3
nb_total_samples = len(y_outcome)

pipeline_smote_under = Pipeline(steps=[('over', SMOTE()), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])
#pipeline_smote_under = Pipeline(steps=[('over', SMOTENC(categorical_features=["fracas_du_bassin", "amputation"])), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])


inner_cv = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=5, random_state=1)

p_grid = {"model__learning_rate": [0.01, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 1], "over__sampling_strategy": [0.1, 0.2, 0.3], "over__k_neighbors":[3,5,8], "under__sampling_strategy":[0.3, 0.5, 0.7]}
clf = GridSearchCV(estimator=pipeline_smote_under, param_grid=p_grid, scoring={'F2':ftwo_scorer}, refit='F2', cv=inner_cv)

outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

nested_scores_smote_undersampling = cross_validate(clf, X_features_imputed, y_outcome, 
                                                   scoring={'F2':ftwo_scorer, 'ROC_AUC':'roc_auc', 'Recall':'recall_macro', 'F1':'f1', 'Brier':"neg_brier_score", 'False_neg_scorer':false_neg_scorer, 'False_pos_scorer':false_pos_scorer}, 
                                                   cv=outer_cv, n_jobs=-1, return_estimator=True, return_indices=True)

print("HistGradientBoostingClassifier with hyperparameter gridsearch")

roc_auc_metric = np.mean(nested_scores_smote_undersampling["test_ROC_AUC"])
roc_auc_metric_std = np.std(nested_scores_smote_undersampling["test_ROC_AUC"])
print(f'AUC (max): {np.round(roc_auc_metric, 2)} +- {np.round(roc_auc_metric_std, 2)}')

f1_score = np.mean(nested_scores_smote_undersampling["test_F1"])
f1_score_std = np.std(nested_scores_smote_undersampling["test_F1"])
print(f'F1 Score (max): {np.round(f1_score, 2)} +- {np.round(f1_score_std, 2)}')

f2_score = np.mean(nested_scores_smote_undersampling["test_F2"])
f2_score_std = np.std(nested_scores_smote_undersampling["test_F2"])
print(f'F2 Score (max): {np.round(f2_score, 2)} +- {np.round(f2_score_std, 2)}')

brier_score = -np.mean(nested_scores_smote_undersampling["test_Brier"])
brier_score_std = -np.std(nested_scores_smote_undersampling["test_Brier"])
print(f'Brier Score (min): {np.round(brier_score, 2)} +- {np.round(brier_score_std, 2)}')

# test_False_neg_scorer returns the number of test false negatives -> to get a % we need to divide by the number of test samples*100
false_neg_score = np.mean(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
false_neg_score_std = np.std(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
print(f'False negative: {int(np.round(false_neg_score, 0))}% +- {int(np.round(false_neg_score_std, 0))}')

false_pos_score = np.mean(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
false_pos_score_std = np.std(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
print(f'False positive: {int(np.round(false_pos_score, 0))}% +- {int(np.round(false_pos_score_std, 0))}')

In [ ]:
for estimator in nested_scores_smote_undersampling['estimator']:
    print(estimator.best_params_)


#### Get individual test set samples prediction vs ground truth

In [ ]:
for i in range(len(nested_scores_smote_undersampling["indices"]["test"])):
    if 469 in nested_scores_smote_undersampling["indices"]["test"][i]:
        print(f'fold={i}')
    #print(f"fold={i}")
    #print(nested_scores_smote_undersampling["indices"]["test"][i])

In [ ]:
fold = 11

y_true_train = np.asarray(y_outcome)[nested_scores_smote_undersampling["indices"]["train"][fold]]
y_true_test = np.asarray(y_outcome)[nested_scores_smote_undersampling["indices"]["test"][fold]]

fitted = nested_scores_smote_undersampling["estimator"][0].fit(X_features_imputed.loc[nested_scores_smote_undersampling["indices"]["train"][fold]], y_true_train) #-#=#

pred_test = fitted.predict(X_features_imputed.loc[nested_scores_smote_undersampling["indices"]["test"][fold]])


In [ ]:
print(f"y_pred: {pred_test}")
print(f"y_true: {y_true_test}")

In [ ]:
# find true positive

true_positive_index = -1

for i in range(len(y_true_test)):
    if y_true_test[i] == 1 and pred_test[i] == 1:
        true_positive_index = nested_scores_smote_undersampling["indices"]["test"][fold][i]
        print(f"true positive at index {true_positive_index}, i={i}")

In [ ]:
# find true negative

true_negative_index = -1

for i in range(len(y_true_test)):
    if y_true_test[i] == 0 and pred_test[i] == 0:
        true_negative_index = nested_scores_smote_undersampling["indices"]["test"][fold][i]
        print(f"true negative at index {true_negative_index}, i={i}")

In [ ]:
# find false negative

false_negative_index = -1

for i in range(len(y_true_test)):
    if y_true_test[i] == 1 and pred_test[i] == 0:
        false_negative_index = nested_scores_smote_undersampling["indices"]["test"][fold][i]
        print(f"false negative at index {false_negative_index}, i={i}")

In [ ]:
# find false positive

false_positive_index = -1

for i in range(len(y_true_test)):
    if y_true_test[i] == 0 and pred_test[i] == 1:
        false_positive_index = nested_scores_smote_undersampling["indices"]["test"][fold][i]
        print(f"false positive at index {false_positive_index}, i={i}")

In [ ]:
# get the data for this individual
print(X_volumes_clinical.loc[469])
#X_volumes_clinical.loc[329].to_csv("patient_inference_results/P0010.csv")

#### SHAP explainer

In [ ]:
print(fitted.best_estimator_["model"])

In [ ]:
explainer = shap.TreeExplainer(fitted.best_estimator_["model"]) 
shap_values2 = explainer(X_features_imputed.loc[nested_scores_smote_undersampling["indices"]["test"][fold]])

In [ ]:
shap.plots.waterfall(shap_values2[94])

#### Confidence intervals with test set bootstrap

In [ ]:
best_params_list = []

for estimator in nested_scores_smote_undersampling["estimator"]:
    best_params_list.append(frozenset(estimator.best_params_.items()))

In [ ]:
from collections import Counter
# Convert dict to a hashable tuple and count occurrences
most_common_params = Counter(best_params_list).most_common(1)[0][0]
final_params = dict(most_common_params)  # Convert back to dict


print("Final chosen hyperparameters:", final_params)

In [ ]:
pipeline = Pipeline(steps=[('over', SMOTE(k_neighbors=3, sampling_strategy=0.3)), ('under', RandomUnderSampler(sampling_strategy=0.7)), ('model', HistGradientBoostingClassifier(learning_rate=0.2))])

In [ ]:
y_pred = cross_val_predict(pipeline, X_features_imputed, y_outcome, cv=20, method='predict_proba')[:,1]

y_pred_binary = [1 if i>=0.5 else 0 for i in y_pred]

In [ ]:
from sklearn.metrics import fbeta_score, confusion_matrix, roc_auc_score, f1_score, brier_score_loss

In [ ]:
rng = np.random.RandomState(seed=12345)
idx = np.arange(len(y_outcome))

y_pred = np.asarray(y_pred)
y_pred_binary = np.asarray(y_pred_binary)
y = np.asarray(y_outcome)
X_dropped = np.asarray(X_features_imputed)

test_roc_auc = []
test_f1 = []
test_ftwos = []
test_brier = []
test_false_neg = []
test_false_pos = []

for i in range(200): # bootstrap with 200 rounds: random sampling with replacement of the predictions

    pred_idx = rng.choice(idx, size=len(idx), replace=True)
    
    roc_auc_test_boot = roc_auc_score(y_score=y_pred[pred_idx], y_true=y[pred_idx])
    f1_test_boot = f1_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx])
    f2_test_boot = fbeta_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx], beta=2)
    brier_test_boot = brier_score_loss(y_proba=y_pred[pred_idx], y_true=y[pred_idx])
    false_neg_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[1,0]
    false_pos_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[0,1]
    
    test_roc_auc.append(roc_auc_test_boot)
    test_f1.append(f1_test_boot)
    test_ftwos.append(f2_test_boot)
    test_brier.append(brier_test_boot)
    test_false_neg.append(false_neg_test_boot/len(idx)*100)
    test_false_pos.append(false_pos_test_boot/len(idx)*100)


In [ ]:
print("Classification performance\n")

bootstrap_roc_auc_test_mean = np.mean(test_roc_auc)
ci_lower = np.percentile(test_roc_auc, 2.5)     # 2.5 percentile (alpha=0.025)
ci_upper = np.percentile(test_roc_auc, 97.5)
print(f"ROC AUC:         {bootstrap_roc_auc_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f1_test_mean = np.mean(test_f1)
ci_lower = np.percentile(test_f1, 2.5)
ci_upper = np.percentile(test_f1, 97.5)
print(f"F1:              {bootstrap_f1_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f2_test_mean = np.mean(test_ftwos)
ci_lower = np.percentile(test_ftwos, 2.5)
ci_upper = np.percentile(test_ftwos, 97.5)
print(f"F2:              {bootstrap_f2_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_brier_test_mean = np.mean(test_brier)
ci_lower = np.percentile(test_brier, 2.5)
ci_upper = np.percentile(test_brier, 97.5)
print(f"Brier loss:      {bootstrap_brier_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_neg_test_mean = np.mean(test_false_neg)
ci_lower = np.percentile(test_false_neg, 2.5)
ci_upper = np.percentile(test_false_neg, 97.5)
print(f"False negatives: {bootstrap_false_neg_test_mean:.2f}%;  95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_pos_test_mean = np.mean(test_false_pos)
ci_lower = np.percentile(test_false_pos, 2.5)
ci_upper = np.percentile(test_false_pos, 97.5)
print(f"False negatives: {bootstrap_false_pos_test_mean:.2f}%; 95% CI {ci_lower:.2f}-{ci_upper:.2f}")

### Gradient boosting with segmentation

In [ ]:
# run --> prepocessing outcome

In [ ]:
X = pd.read_csv(DATA_DIRECTORY+"segmentation_data_anonymized.csv", usecols=range(1,16))
X.head()

y = get_mortality_6m()

# Identify indexes where 'mortality' is NaN in y
try:
    nan_indexes = y.loc[pd.isna(y["mortality"]), :].index
except:
    nan_indexes=[]
print(f"Indexes with NaN in 'mortality': {nan_indexes}")

# Drop rows with NaN in y
y = y.drop(index=nan_indexes)

# Ensure 'name' is the index or used for alignment
X = X[~X['name'].isin(y.loc[nan_indexes, 'name'])]

# Verify the shapes after cleaning
print(f"Shape of y after cleaning: {y.shape}")
print(f"Shape of X after cleaning: {X.shape}")

# Vérifiez les valeurs uniques de 'name' dans y
print("Unique 'name' values in y:", y['name'].nunique())

# Vérifiez les valeurs uniques de 'name' dans X
print("Unique 'name' values in X:", X['name'].nunique())

# Noms dans y mais pas dans X
missing_in_X = set(y['name']) - set(X['name'])
print(f"Names in y but not in X: {missing_in_X}")

# Noms dans X mais pas dans y
missing_in_y = set(X['name']) - set(y['name'])
print(f"Names in X but not in y: {missing_in_y}")

# Trouver les noms communs
common_names = set(y['name']).intersection(set(X['name']))

# Filtrer y et X pour ne garder que les noms communs
y = y[y['name'].isin(common_names)]
X = X[X['name'].isin(common_names)]

# Vérifiez les dimensions après nettoyage
print(f"Shape of y after alignment: {y.shape}")
print(f"Shape of X after alignment: {X.shape}")

# Check for duplicates in 'name' in y
print("Number of duplicate names in y:", y['name'].duplicated().sum())

# Check for duplicates in 'name' in X
print("Number of duplicate names in X:", X['name'].duplicated().sum())

# Remove duplicates from both DataFrames
y = y.drop_duplicates(subset=['name'])
X = X.drop_duplicates(subset=['name'])

# Verify the shapes again
print(f"Shape of y after removing duplicates: {y.shape}")
print(f"Shape of X after removing duplicates: {X.shape}")

# Check if 'name' values are the same in both DataFrames
common_names = set(y['name']) == set(X['name'])
print(f"Are 'name' values aligned between y and X {common_names}")

# Trier y et X par la colonne 'name'
y = y.sort_values(by='name').reset_index(drop=True)
X = X.sort_values(by='name').reset_index(drop=True)

# Vérifier si les noms sont alignés
print((y['name'].values == X['name'].values).all())

# Drop the 'name' column from X
X_features = X.drop(columns=['name'])

# imputation with median 
X_features_imputed = X_features.fillna(X_features.median())

# Drop the 'name' column from y (if it exists)
y_outcome = y.drop(columns=['name'])

# Ensure y_outcome is a 1D array
y_outcome = y_outcome.values.ravel()  # Convert to 1D if using pandas DataFrame

In [ ]:
num_ones = np.sum(y_outcome == 1)  # True values are treated as 1 in numpy
print(f"Number of y = 1 in y_outcome: {num_ones}")

In [ ]:
X_features_imputed.head()

In [ ]:
FOLDS = 5
N_REPEATS = 3
nb_total_samples = len(y_outcome)

pipeline_smote_under = Pipeline(steps=[('over', SMOTE()), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])
#pipeline_smote_under = Pipeline(steps=[('over', SMOTENC(categorical_features=["fracas_du_bassin", "amputation"])), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])


inner_cv = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=5, random_state=1)

p_grid = {"model__learning_rate": [0.01, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 1], "over__sampling_strategy": [0.1, 0.2, 0.3], "over__k_neighbors":[3,5,8], "under__sampling_strategy":[0.3, 0.5, 0.7]}
clf = GridSearchCV(estimator=pipeline_smote_under, param_grid=p_grid, scoring={'F2':ftwo_scorer}, refit='F2', cv=inner_cv)

outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

nested_scores_smote_undersampling = cross_validate(clf, X_features_imputed, y_outcome, 
                                                   scoring={'F2':ftwo_scorer, 'ROC_AUC':'roc_auc', 'Recall':'recall_macro', 'F1':'f1', 'Brier':"neg_brier_score", 'False_neg_scorer':false_neg_scorer, 'False_pos_scorer':false_pos_scorer}, 
                                                   cv=outer_cv, n_jobs=-1, return_estimator=True, return_indices=True)

print("HistGradientBoostingClassifier with hyperparameter gridsearch")

roc_auc_metric = np.mean(nested_scores_smote_undersampling["test_ROC_AUC"])
roc_auc_metric_std = np.std(nested_scores_smote_undersampling["test_ROC_AUC"])
print(f'AUC (max): {np.round(roc_auc_metric, 2)} +- {np.round(roc_auc_metric_std, 2)}')

f1_score = np.mean(nested_scores_smote_undersampling["test_F1"])
f1_score_std = np.std(nested_scores_smote_undersampling["test_F1"])
print(f'F1 Score (max): {np.round(f1_score, 2)} +- {np.round(f1_score_std, 2)}')

f2_score = np.mean(nested_scores_smote_undersampling["test_F2"])
f2_score_std = np.std(nested_scores_smote_undersampling["test_F2"])
print(f'F2 Score (max): {np.round(f2_score, 2)} +- {np.round(f2_score_std, 2)}')

brier_score = -np.mean(nested_scores_smote_undersampling["test_Brier"])
brier_score_std = -np.std(nested_scores_smote_undersampling["test_Brier"])
print(f'Brier Score (min): {np.round(brier_score, 2)} +- {np.round(brier_score_std, 2)}')

# test_False_neg_scorer returns the number of test false negatives -> to get a % we need to divide by the number of test samples*100
false_neg_score = np.mean(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
false_neg_score_std = np.std(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
print(f'False negative: {int(np.round(false_neg_score, 0))}% +- {int(np.round(false_neg_score_std, 0))}')

false_pos_score = np.mean(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
false_pos_score_std = np.std(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
print(f'False positive: {int(np.round(false_pos_score, 0))}% +- {int(np.round(false_pos_score_std, 0))}')

#### Confidence intervals with test set bootstrap

In [ ]:
best_params_list = []

for estimator in nested_scores_smote_undersampling["estimator"]:
    best_params_list.append(frozenset(estimator.best_params_.items()))

In [ ]:
from collections import Counter
# Convert dict to a hashable tuple and count occurrences
most_common_params = Counter(best_params_list).most_common(1)[0][0]
final_params = dict(most_common_params)  # Convert back to dict


print("Final chosen hyperparameters:", final_params)

In [ ]:
pipeline = Pipeline(steps=[('over', SMOTE(k_neighbors=3, sampling_strategy=0.3)), ('under', RandomUnderSampler(sampling_strategy=0.7)), ('model', HistGradientBoostingClassifier(learning_rate=0.2))])

In [ ]:
y_pred = cross_val_predict(pipeline, X_features_imputed, y_outcome, cv=20, method='predict_proba')[:,1]

y_pred_binary = [1 if i>=0.5 else 0 for i in y_pred]

In [ ]:
from sklearn.metrics import fbeta_score, confusion_matrix, roc_auc_score, f1_score, brier_score_loss

In [ ]:
rng = np.random.RandomState(seed=12345)
idx = np.arange(len(y_outcome))

y_pred = np.asarray(y_pred)
y_pred_binary = np.asarray(y_pred_binary)
y = np.asarray(y_outcome)
X_dropped = np.asarray(X_features_imputed)

test_roc_auc = []
test_f1 = []
test_ftwos = []
test_brier = []
test_false_neg = []
test_false_pos = []

for i in range(200): # bootstrap with 200 rounds: random sampling with replacement of the predictions

    pred_idx = rng.choice(idx, size=len(idx), replace=True)
    
    roc_auc_test_boot = roc_auc_score(y_score=y_pred[pred_idx], y_true=y[pred_idx])
    f1_test_boot = f1_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx])
    f2_test_boot = fbeta_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx], beta=2)
    brier_test_boot = brier_score_loss(y_proba=y_pred[pred_idx], y_true=y[pred_idx])
    false_neg_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[1,0]
    false_pos_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[0,1]
    
    test_roc_auc.append(roc_auc_test_boot)
    test_f1.append(f1_test_boot)
    test_ftwos.append(f2_test_boot)
    test_brier.append(brier_test_boot)
    test_false_neg.append(false_neg_test_boot/len(idx)*100)
    test_false_pos.append(false_pos_test_boot/len(idx)*100)


In [ ]:
print("Classification performance\n")

bootstrap_roc_auc_test_mean = np.mean(test_roc_auc)
ci_lower = np.percentile(test_roc_auc, 2.5)     # 2.5 percentile (alpha=0.025)
ci_upper = np.percentile(test_roc_auc, 97.5)
print(f"ROC AUC:         {bootstrap_roc_auc_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f1_test_mean = np.mean(test_f1)
ci_lower = np.percentile(test_f1, 2.5)
ci_upper = np.percentile(test_f1, 97.5)
print(f"F1:              {bootstrap_f1_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f2_test_mean = np.mean(test_ftwos)
ci_lower = np.percentile(test_ftwos, 2.5)
ci_upper = np.percentile(test_ftwos, 97.5)
print(f"F2:              {bootstrap_f2_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_brier_test_mean = np.mean(test_brier)
ci_lower = np.percentile(test_brier, 2.5)
ci_upper = np.percentile(test_brier, 97.5)
print(f"Brier loss:      {bootstrap_brier_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_neg_test_mean = np.mean(test_false_neg)
ci_lower = np.percentile(test_false_neg, 2.5)
ci_upper = np.percentile(test_false_neg, 97.5)
print(f"False negatives: {bootstrap_false_neg_test_mean:.2f}%;  95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_pos_test_mean = np.mean(test_false_pos)
ci_lower = np.percentile(test_false_pos, 2.5)
ci_upper = np.percentile(test_false_pos, 97.5)
print(f"False negatives: {bootstrap_false_pos_test_mean:.2f}%; 95% CI {ci_lower:.2f}-{ci_upper:.2f}")

#### Get individual test set samples prediction vs ground truth

In [ ]:
fold = 0
pred = nested_scores_smote_undersampling["estimator"][0].predict(X_features_imputed.loc[nested_scores_smote_undersampling["indices"]["test"][fold]])
y_true = np.asarray(y_outcome)[nested_scores_smote_undersampling["indices"]["test"][fold]]

In [ ]:
print(f"y_pred: {pred}")
print(f"y_true: {y_true}")

In [ ]:
# find false positive

false_negative_index = -1

for i in range(len(y_true)):
    if y_true[i] == 1 and pred[i] == 0:
        false_negative_index = nested_scores_smote_undersampling["indices"]["test"][fold][i]
        print(f"false negative at index {false_negative_index}")

In [ ]:
# get the data for this individual
print(X.loc[false_negative_index])
X.loc[false_negative_index].to_csv("patient_inference_results/P0501.csv")

In [ ]:
# find false positive

false_positive_index = -1

for i in range(len(y_true)):
    if y_true[i] == 0 and pred[i] == 1:
        false_positive_index = nested_scores_smote_undersampling["indices"]["test"][fold][i]
        print(f"false positive at index {false_positive_index}")

In [ ]:
# get the data for this individual
print(X.loc[false_positive_index])
#X.loc[false_positive_index].to_csv("patient_inference_results/P0498.csv")

### Gradient boosting with traumatrix model

In [ ]:
X = pd.read_csv(DATA_DIRECTORY+"segmentation_data_anonymized.csv", usecols=[1]+list(range(16,31)))
X.head()

In [ ]:


y = get_mortality_6m()

# Identify indexes where 'mortality' is NaN in y
try:
    nan_indexes = y.loc[pd.isna(y["mortality"]), :].index
except:
    nan_indexes=[]
print(f"Indexes with NaN in 'mortality': {nan_indexes}")

# Drop rows with NaN in y
y = y.drop(index=nan_indexes)

# Ensure 'name' is the index or used for alignment
X = X[~X['name'].isin(y.loc[nan_indexes, 'name'])]

# Verify the shapes after cleaning
print(f"Shape of y after cleaning: {y.shape}")
print(f"Shape of X after cleaning: {X.shape}")

# Vérifiez les valeurs uniques de 'name' dans y
print("Unique 'name' values in y:", y['name'].nunique())

# Vérifiez les valeurs uniques de 'name' dans X
print("Unique 'name' values in X:", X['name'].nunique())

# Noms dans y mais pas dans X
missing_in_X = set(y['name']) - set(X['name'])
print(f"Names in y but not in X: {missing_in_X}")

# Noms dans X mais pas dans y
missing_in_y = set(X['name']) - set(y['name'])
print(f"Names in X but not in y: {missing_in_y}")

# Trouver les noms communs
common_names = set(y['name']).intersection(set(X['name']))

# Filtrer y et X pour ne garder que les noms communs
y = y[y['name'].isin(common_names)]
X = X[X['name'].isin(common_names)]

# Vérifiez les dimensions après nettoyage
print(f"Shape of y after alignment: {y.shape}")
print(f"Shape of X after alignment: {X.shape}")

# Check for duplicates in 'name' in y
print("Number of duplicate names in y:", y['name'].duplicated().sum())

# Check for duplicates in 'name' in X
print("Number of duplicate names in X:", X['name'].duplicated().sum())

# Remove duplicates from both DataFrames
y = y.drop_duplicates(subset=['name'])
X = X.drop_duplicates(subset=['name'])

# Verify the shapes again
print(f"Shape of y after removing duplicates: {y.shape}")
print(f"Shape of X after removing duplicates: {X.shape}")

# Check if 'name' values are the same in both DataFrames
common_names = set(y['name']) == set(X['name'])
print(f"Are 'name' values aligned between y and X {common_names}")

# Trier y et X par la colonne 'name'
y = y.sort_values(by='name').reset_index(drop=True)
X = X.sort_values(by='name').reset_index(drop=True)

# Vérifier si les noms sont alignés
print((y['name'].values == X['name'].values).all())

# Drop the 'name' column from X
X_features = X.drop(columns=['name'])

# imputation with median 
X_features_imputed = X_features.fillna(X_features.median())

# Drop the 'name' column from y (if it exists)
y_outcome = y.drop(columns=['name'])

# Ensure y_outcome is a 1D array
y_outcome = y_outcome.values.ravel()  # Convert to 1D if using pandas DataFrame

In [ ]:
pipeline_smote_under_traumatrix = Pipeline(steps=[('over', SMOTE()), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])
#pipeline_smote_under = Pipeline(steps=[('over', SMOTENC(categorical_features=["fracas_du_bassin", "amputation"])), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])


inner_cv_traumatrix = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=5, random_state=1)

p_grid = {"model__learning_rate": [0.01, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 1], "over__sampling_strategy": [0.1, 0.2, 0.3], "over__k_neighbors":[3,5,8], "under__sampling_strategy":[0.3, 0.5, 0.7]}
clf_traumatrix = GridSearchCV(estimator=pipeline_smote_under_traumatrix, param_grid=p_grid, scoring={'F2':ftwo_scorer}, refit='F2', cv=inner_cv_traumatrix)

outer_cv_traumatrix = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

nested_scores_smote_undersampling_traumatrix = cross_validate(clf_traumatrix, X_features_imputed, y_outcome, 
                                                   scoring={'F2':ftwo_scorer, 'ROC_AUC':'roc_auc', 'Recall':'recall_macro', 'F1':'f1', 'Brier':"neg_brier_score", 'False_neg_scorer':false_neg_scorer, 'False_pos_scorer':false_pos_scorer}, 
                                                   cv=outer_cv_traumatrix, n_jobs=-1, return_estimator=True, return_indices=True)

print("HistGradientBoostingClassifier with hyperparameter gridsearch")

roc_auc_metric = np.mean(nested_scores_smote_undersampling_traumatrix["test_ROC_AUC"])
roc_auc_metric_std = np.std(nested_scores_smote_undersampling_traumatrix["test_ROC_AUC"])
print(f'AUC (max): {np.round(roc_auc_metric, 2)} +- {np.round(roc_auc_metric_std, 2)}')

f1_score = np.mean(nested_scores_smote_undersampling_traumatrix["test_F1"])
f1_score_std = np.std(nested_scores_smote_undersampling_traumatrix["test_F1"])
print(f'F1 Score (max): {np.round(f1_score, 2)} +- {np.round(f1_score_std, 2)}')

f2_score = np.mean(nested_scores_smote_undersampling_traumatrix["test_F2"])
f2_score_std = np.std(nested_scores_smote_undersampling_traumatrix["test_F2"])
print(f'F2 Score (max): {np.round(f2_score, 2)} +- {np.round(f2_score_std, 2)}')

brier_score = -np.mean(nested_scores_smote_undersampling_traumatrix["test_Brier"])
brier_score_std = -np.std(nested_scores_smote_undersampling_traumatrix["test_Brier"])
print(f'Brier Score (min): {np.round(brier_score, 2)} +- {np.round(brier_score_std, 2)}')

# test_False_neg_scorer returns the number of test false negatives -> to get a % we need to divide by the number of test samples*100
false_neg_score = np.mean(nested_scores_smote_undersampling_traumatrix["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
false_neg_score_std = np.std(nested_scores_smote_undersampling_traumatrix["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
print(f'False negative: {int(np.round(false_neg_score, 0))}% +- {int(np.round(false_neg_score_std, 0))}')

false_pos_score = np.mean(nested_scores_smote_undersampling_traumatrix["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
false_pos_score_std = np.std(nested_scores_smote_undersampling_traumatrix["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
print(f'False positive: {int(np.round(false_pos_score, 0))}% +- {int(np.round(false_pos_score_std, 0))}')

#### Get individual test set samples prediction vs ground truth

In [ ]:
fold = 2

y_true_train_traumatrix = np.asarray(y_outcome)[nested_scores_smote_undersampling_traumatrix["indices"]["train"][fold]]
y_true_test_traumatrix = np.asarray(y_outcome)[nested_scores_smote_undersampling_traumatrix["indices"]["test"][fold]]

fitted_traumatrix = nested_scores_smote_undersampling_traumatrix["estimator"][0].fit(X_features_imputed.loc[nested_scores_smote_undersampling_traumatrix["indices"]["train"][fold]], y_true_train) #-#=#

pred_test_traumatrix = fitted_traumatrix.predict(X_features_imputed.loc[nested_scores_smote_undersampling_traumatrix["indices"]["test"][fold]])


In [ ]:
print(f"y_pred: {pred_test_traumatrix}")
print(f"y_true: {y_true_test_traumatrix}")

In [ ]:
# find true positive

true_positive_index_traumatrix = -1

for i in range(len(y_true_test_traumatrix)):
    if y_true_test_traumatrix[i] == 1 and pred_test_traumatrix[i] == 1:
        true_positive_index_traumatrix = nested_scores_smote_undersampling["indices"]["test"][fold][i]
        print(f"true positive at index {true_positive_index_traumatrix}, i={i}")

In [ ]:
# find true negative

true_negative_index_traumatrix = -1

for i in range(len(y_true_test_traumatrix)):
    if y_true_test_traumatrix[i] == 0 and pred_test_traumatrix[i] == 0:
        true_negative_index_traumatrix = nested_scores_smote_undersampling_traumatrix["indices"]["test"][fold][i]
        print(f"true negative at index {true_negative_index_traumatrix}, i={i}")

In [ ]:
# find false negative

false_negative_index_traumatrix = -1

for i in range(len(y_true_test_traumatrix)):
    if y_true_test_traumatrix[i] == 1 and pred_test_traumatrix[i] == 0:
        false_negative_index_traumatrix = nested_scores_smote_undersampling_traumatrix["indices"]["test"][fold][i]
        print(f"false negative at index {false_negative_index_traumatrix}, i={i}")

In [ ]:
# find false positive

false_positive_index_traumatrix = -1

for i in range(len(y_true_test_traumatrix)):
    if y_true_test_traumatrix[i] == 0 and pred_test_traumatrix[i] == 1:
        false_positive_index_traumatrix = nested_scores_smote_undersampling_traumatrix["indices"]["test"][fold][i]
        print(f"false positive at index {false_positive_index_traumatrix}, i={i}")

In [ ]:
# get the data for this individual
print(X_volumes_clinical.loc[24])
#X_volumes_clinical.loc[329].to_csv("patient_inference_results/P0010.csv")

#### SHAP explainer

In [ ]:
print(fitted.best_estimator_["model"])

In [ ]:
explainer = shap.TreeExplainer(fitted.best_estimator_["model"]) 
shap_values2 = explainer(X_features_imputed.loc[nested_scores_smote_undersampling["indices"]["test"][fold]])

In [ ]:
shap.plots.waterfall(shap_values2[0])

#### Confidence intervals with test set bootstrap

In [ ]:
best_params_list = []

for estimator in nested_scores_smote_undersampling["estimator"]:
    best_params_list.append(frozenset(estimator.best_params_.items()))

In [ ]:
from collections import Counter
# Convert dict to a hashable tuple and count occurrences
most_common_params = Counter(best_params_list).most_common(1)[0][0]
final_params = dict(most_common_params)  # Convert back to dict


print("Final chosen hyperparameters:", final_params)

In [ ]:
pipeline = Pipeline(steps=[('over', SMOTE(k_neighbors=3, sampling_strategy=0.3)), ('under', RandomUnderSampler(sampling_strategy=0.7)), ('model', HistGradientBoostingClassifier(learning_rate=0.2))])

In [ ]:
y_pred = cross_val_predict(pipeline, X_features_imputed, y_outcome, cv=20, method='predict_proba')[:,1]

y_pred_binary = [1 if i>=0.5 else 0 for i in y_pred]

In [ ]:
from sklearn.metrics import fbeta_score, confusion_matrix, roc_auc_score, f1_score, brier_score_loss

In [ ]:
rng = np.random.RandomState(seed=12345)
idx = np.arange(len(y_outcome))

y_pred = np.asarray(y_pred)
y_pred_binary = np.asarray(y_pred_binary)
y = np.asarray(y_outcome)
X_dropped = np.asarray(X_features_imputed)

test_roc_auc = []
test_f1 = []
test_ftwos = []
test_brier = []
test_false_neg = []
test_false_pos = []

for i in range(200): # bootstrap with 200 rounds: random sampling with replacement of the predictions

    pred_idx = rng.choice(idx, size=len(idx), replace=True)
    
    roc_auc_test_boot = roc_auc_score(y_score=y_pred[pred_idx], y_true=y[pred_idx])
    f1_test_boot = f1_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx])
    f2_test_boot = fbeta_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx], beta=2)
    brier_test_boot = brier_score_loss(y_proba=y_pred[pred_idx], y_true=y[pred_idx])
    false_neg_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[1,0]
    false_pos_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[0,1]
    
    test_roc_auc.append(roc_auc_test_boot)
    test_f1.append(f1_test_boot)
    test_ftwos.append(f2_test_boot)
    test_brier.append(brier_test_boot)
    test_false_neg.append(false_neg_test_boot/len(idx)*100)
    test_false_pos.append(false_pos_test_boot/len(idx)*100)


In [ ]:
print("Classification performance\n")

bootstrap_roc_auc_test_mean = np.mean(test_roc_auc)
ci_lower = np.percentile(test_roc_auc, 2.5)     # 2.5 percentile (alpha=0.025)
ci_upper = np.percentile(test_roc_auc, 97.5)
print(f"ROC AUC:         {bootstrap_roc_auc_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f1_test_mean = np.mean(test_f1)
ci_lower = np.percentile(test_f1, 2.5)
ci_upper = np.percentile(test_f1, 97.5)
print(f"F1:              {bootstrap_f1_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f2_test_mean = np.mean(test_ftwos)
ci_lower = np.percentile(test_ftwos, 2.5)
ci_upper = np.percentile(test_ftwos, 97.5)
print(f"F2:              {bootstrap_f2_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_brier_test_mean = np.mean(test_brier)
ci_lower = np.percentile(test_brier, 2.5)
ci_upper = np.percentile(test_brier, 97.5)
print(f"Brier loss:      {bootstrap_brier_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_neg_test_mean = np.mean(test_false_neg)
ci_lower = np.percentile(test_false_neg, 2.5)
ci_upper = np.percentile(test_false_neg, 97.5)
print(f"False negatives: {bootstrap_false_neg_test_mean:.2f}%;  95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_pos_test_mean = np.mean(test_false_pos)
ci_lower = np.percentile(test_false_pos, 2.5)
ci_upper = np.percentile(test_false_pos, 97.5)
print(f"False negatives: {bootstrap_false_pos_test_mean:.2f}%; 95% CI {ci_lower:.2f}-{ci_upper:.2f}")

#### Confidence intervals with test set bootstrap

In [ ]:
best_params_list_traumatrix = []

for estimator in nested_scores_smote_undersampling_traumatrix["estimator"]:
    best_params_list_traumatrix.append(frozenset(estimator.best_params_.items()))

In [ ]:
from collections import Counter
# Convert dict to a hashable tuple and count occurrences
most_common_params_traumatrix = Counter(best_params_list_traumatrix).most_common(1)[0][0]
final_params_traumatrix = dict(most_common_params_traumatrix)  # Convert back to dict


print("Final chosen hyperparameters:", final_params_traumatrix)

In [ ]:
pipeline_traumatrix = Pipeline(steps=[('over', SMOTE(k_neighbors=8, sampling_strategy=0.2)), ('under', RandomUnderSampler(sampling_strategy=0.7)), ('model', HistGradientBoostingClassifier(learning_rate=0.08))])

In [ ]:
y_pred_traumatrix = cross_val_predict(pipeline_traumatrix, X_features_imputed, y_outcome, cv=20, method='predict_proba')[:,1]

y_pred_binary_traumatrix = [1 if i>=0.5 else 0 for i in y_pred_traumatrix]

In [ ]:
from sklearn.metrics import fbeta_score, confusion_matrix, roc_auc_score, f1_score, brier_score_loss

In [ ]:
rng = np.random.RandomState(seed=12345)
idx = np.arange(len(y_outcome))

y_pred_traumatrix = np.asarray(y_pred_traumatrix)
y_pred_binary_traumatrix = np.asarray(y_pred_binary_traumatrix)
y = np.asarray(y_outcome)
X_dropped = np.asarray(X_features_imputed)

test_roc_auc = []
test_f1 = []
test_ftwos = []
test_brier = []
test_false_neg = []
test_false_pos = []

for i in range(200): # bootstrap with 200 rounds: random sampling with replacement of the predictions

    pred_idx = rng.choice(idx, size=len(idx), replace=True)
    
    roc_auc_test_boot = roc_auc_score(y_score=y_pred_traumatrix[pred_idx], y_true=y[pred_idx])
    f1_test_boot = f1_score(y_pred=y_pred_binary_traumatrix[pred_idx], y_true=y[pred_idx])
    f2_test_boot = fbeta_score(y_pred=y_pred_binary_traumatrix[pred_idx], y_true=y[pred_idx], beta=2)
    brier_test_boot = brier_score_loss(y_proba=y_pred_traumatrix[pred_idx], y_true=y[pred_idx])
    false_neg_test_boot = confusion_matrix(y[pred_idx], y_pred_binary_traumatrix[pred_idx])[1,0]
    false_pos_test_boot = confusion_matrix(y[pred_idx], y_pred_binary_traumatrix[pred_idx])[0,1]
    
    test_roc_auc.append(roc_auc_test_boot)
    test_f1.append(f1_test_boot)
    test_ftwos.append(f2_test_boot)
    test_brier.append(brier_test_boot)
    test_false_neg.append(false_neg_test_boot/len(idx)*100)
    test_false_pos.append(false_pos_test_boot/len(idx)*100)


In [ ]:
print("Classification performance\n")

bootstrap_roc_auc_test_mean = np.mean(test_roc_auc)
ci_lower = np.percentile(test_roc_auc, 2.5)     # 2.5 percentile (alpha=0.025)
ci_upper = np.percentile(test_roc_auc, 97.5)
print(f"ROC AUC:         {bootstrap_roc_auc_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f1_test_mean = np.mean(test_f1)
ci_lower = np.percentile(test_f1, 2.5)
ci_upper = np.percentile(test_f1, 97.5)
print(f"F1:              {bootstrap_f1_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f2_test_mean = np.mean(test_ftwos)
ci_lower = np.percentile(test_ftwos, 2.5)
ci_upper = np.percentile(test_ftwos, 97.5)
print(f"F2:              {bootstrap_f2_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_brier_test_mean = np.mean(test_brier)
ci_lower = np.percentile(test_brier, 2.5)
ci_upper = np.percentile(test_brier, 97.5)
print(f"Brier loss:      {bootstrap_brier_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_neg_test_mean = np.mean(test_false_neg)
ci_lower = np.percentile(test_false_neg, 2.5)
ci_upper = np.percentile(test_false_neg, 97.5)
print(f"False negatives: {bootstrap_false_neg_test_mean:.2f}%;  95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_pos_test_mean = np.mean(test_false_pos)
ci_lower = np.percentile(test_false_pos, 2.5)
ci_upper = np.percentile(test_false_pos, 97.5)
print(f"False negatives: {bootstrap_false_pos_test_mean:.2f}%; 95% CI {ci_lower:.2f}-{ci_upper:.2f}")

### Gradient boosting with all prehospital features

In [4]:
# run preprocessing outcome mortality

In [5]:

data_full = pd.read_csv(DATA_DIRECTORY+"final_database_clinical_segmentation.csv")
data_full.head()

,Unnamed: 0,Unnamed: 0_x,name,Date Naissance,Age,Sexe,Venue,Entrée venue,Unnamed: 9,Exclusion,...,score_glasgow_initial,score_glasgow_moteur_initial,anomalie_pupillaire_prehospitalier,frequence_cardiaque_FC_arrivee_du_smur,arret_cardio_respiratoire_massage,penetrant_objet,ischemie_du_membre,hemorragie_externe,amputation,outcome_neurochir_pic
0,0,0,P0000,11/19/1941,NaN,M,2.000010e+12,8/5/2020 21:36,NaN,NaN,...,15.0,6.0,0.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,P0001,11/21/1968,NaN,M,2.000010e+12,8/6/2020 11:50,NaN,NaN,...,15.0,6.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,P0002,6/14/1997,NaN,M,2.000010e+12,8/7/2020 21:31,NaN,NaN,...,15.0,6.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,P0003,8/5/1978,NaN,F,2.000010e+12,8/8/2020 19:57,NaN,NaN,...,14.0,6.0,0.0,120.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,P0004,11/17/1986,NaN,M,2.000010e+12,8/9/2020 2:19,NaN,NaN,...,15.0,6.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Select columns 16 to 36
X_prehosp = data_full.iloc[:, 15:35]

# Convert all columns to numeric, replacing non-numeric values and empty strings with NaN
X_prehosp = X_prehosp.apply(pd.to_numeric, errors='coerce')

# Include column name
X_prehosp = pd.concat([X_prehosp, data_full.iloc[:, 2]], axis=1)

In [7]:



# Verify if there are NaN values
print(f"Number of NaN values per column:\n{X_prehosp.isna().sum()}")

# Display the resulting DataFrame
X_prehosp.head()

Number of NaN values per column:
PAS  SMUR                   40
PAD  SMUR                   49
FC SMUR                     42
FR SMUR                    485
Shock Index SMUR            55
GCS SMUR                    17
GCS (M) SMUR                30
Hémocue SMUR               243
Shock Index inversé         54
Shock index diastolique     62
Anomalie pupille SMUR       17
Fracas bassin               14
Amputation                  12
ACR SMUR                    11
Hémorragie ext SMUR         12
Ischémie                    12
Intubation prehosp          11
Expansion volémique         20
OsmoTH prehosp              11
Vasopresseur prehosp        12
name                         0
dtype: int64


,PAS SMUR,PAD SMUR,FC SMUR,FR SMUR,Shock Index SMUR,GCS SMUR,GCS (M) SMUR,Hémocue SMUR,Shock Index inversé,Shock index diastolique,...,Fracas bassin,Amputation,ACR SMUR,Hémorragie ext SMUR,Ischémie,Intubation prehosp,Expansion volémique,OsmoTH prehosp,Vasopresseur prehosp,name
0,190.0,103.0,137.0,NaN,0.72,15.0,6.0,NaN,1.39,1.33,...,0.0,0.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,P0000
1,87.0,49.0,56.0,NaN,0.64,15.0,6.0,NaN,1.55,1.14,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,0.0,0.0,P0001
2,100.0,60.0,100.0,17.0,1.00,15.0,6.0,NaN,1.00,1.67,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,0.0,0.0,P0002
3,101.0,64.0,120.0,NaN,1.19,14.0,6.0,13.1,0.84,1.88,...,0.0,0.0,0.0,0.0,0.0,0.0,250.0,0.0,0.0,P0003
4,110.0,71.0,107.0,18.0,0.97,15.0,6.0,15.8,1.03,1.51,...,0.0,0.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,P0004


In [ ]:
X = X_prehosp.copy()
y = get_mortality_6m()

In [9]:
# Identify indexes where 'mortality' is NaN in y
nan_indexes = y.loc[pd.isna(y["mortality"]), :].index
print(f"Indexes with NaN in 'mortality': {nan_indexes}")

# Drop rows with NaN in y
y = y.drop(index=nan_indexes)

# Ensure 'name' is the index or used for alignment
X = X[~X['name'].isin(y.loc[nan_indexes, 'name'])]

# Verify the shapes after cleaning
print(f"Shape of y after cleaning: {y.shape}")
print(f"Shape of X after cleaning: {X.shape}")

# Vérifiez les valeurs uniques de 'name' dans y
print("Unique 'name' values in y:", y['name'].nunique())

# Vérifiez les valeurs uniques de 'name' dans X
print("Unique 'name' values in X:", X['name'].nunique())

# Noms dans y mais pas dans X
missing_in_X = set(y['name']) - set(X['name'])
print(f"Names in y but not in X: {missing_in_X}")

# Noms dans X mais pas dans y
missing_in_y = set(X['name']) - set(y['name'])
print(f"Names in X but not in y: {missing_in_y}")

# Trouver les noms communs
common_names = set(y['name']).intersection(set(X['name']))

# Filtrer y et X pour ne garder que les noms communs
y = y[y['name'].isin(common_names)]
X = X[X['name'].isin(common_names)]

# Vérifiez les dimensions après nettoyage
print(f"Shape of y after alignment: {y.shape}")
print(f"Shape of X after alignment: {X.shape}")

# Check for duplicates in 'name' in y
print("Number of duplicate names in y:", y['name'].duplicated().sum())

# Check for duplicates in 'name' in X
print("Number of duplicate names in X:", X['name'].duplicated().sum())

# Remove duplicates from both DataFrames
y = y.drop_duplicates(subset=['name'])
X = X.drop_duplicates(subset=['name'])

# Verify the shapes again
print(f"Shape of y after removing duplicates: {y.shape}")
print(f"Shape of X after removing duplicates: {X.shape}")

# Check if 'name' values are the same in both DataFrames
common_names = set(y['name']) == set(X['name'])
print(f"Are 'name' values aligned between y and X {common_names}")

# Drop the 'name' column from X_volumes_clinical_with_outcome
X_features = X.drop(columns=['name'])

# imputation with median 
X_features_imputed = X_features.fillna(X_features.median())

# Drop the 'name' column from y (if it exists)
y_outcome = y.drop(columns=['name'])

# Ensure y_outcome is a 1D array
y_outcome = y_outcome.values.ravel()  # Convert to 1D if using pandas DataFrame

FOLDS = 5
N_REPEATS = 3
nb_total_samples = len(y_outcome)

Indexes with NaN in 'mortality': Index([], dtype='int64')
Shape of y after cleaning: (566, 2)
Shape of X after cleaning: (534, 21)
Unique 'name' values in y: 562
Unique 'name' values in X: 534
Names in y but not in X: {'P0349', 'P0518', 'P0155', 'P0150', 'P0383', 'P0224', 'P0217', 'P0564', 'P0532', 'P0327', 'P0147', 'P0207', 'P0018', 'P0195', 'P0430', 'P0239', 'P0149', 'P0480', 'P0083', 'P0369', 'P0422', 'P0412', 'P0111', 'P0356', 'P0010', 'P0527', 'P0343', 'P0491'}
Names in X but not in y: set()
Shape of y after alignment: (536, 2)
Shape of X after alignment: (534, 21)
Number of duplicate names in y: 2
Number of duplicate names in X: 0
Shape of y after removing duplicates: (534, 2)
Shape of X after removing duplicates: (534, 21)
Are 'name' values aligned between y and X True


In [10]:
ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [11]:

pipeline_smote_under = Pipeline(steps=[('over', SMOTE()), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])
#pipeline_smote_under = Pipeline(steps=[('over', SMOTENC(categorical_features=["fracas_du_bassin", "amputation"])), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])


inner_cv = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=5, random_state=1)

p_grid = {"model__learning_rate": [0.01, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 1], "over__sampling_strategy": [0.1, 0.2, 0.3], "over__k_neighbors":[3,5,8], "under__sampling_strategy":[0.3, 0.5, 0.7]}
clf = GridSearchCV(estimator=pipeline_smote_under, param_grid=p_grid, scoring={'F2':ftwo_scorer}, refit='F2', cv=inner_cv)

outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

nested_scores_smote_undersampling = cross_validate(clf, X_features_imputed, y_outcome, 
                                                scoring={'F2':ftwo_scorer, 'ROC_AUC':'roc_auc', 'Recall':'recall_macro', 'F1':'f1', 'Brier':"neg_brier_score", 'False_neg_scorer':false_neg_scorer, 'False_pos_scorer':false_pos_scorer}, 
                                                cv=outer_cv, n_jobs=-1, return_estimator=True, return_indices=True)

print("HistGradientBoostingClassifier with hyperparameter gridsearch")

roc_auc_metric = np.mean(nested_scores_smote_undersampling["test_ROC_AUC"])
roc_auc_metric_std = np.std(nested_scores_smote_undersampling["test_ROC_AUC"])
print(f'AUC (max): {np.round(roc_auc_metric, 2)} +- {np.round(roc_auc_metric_std, 2)}')

f1_score = np.mean(nested_scores_smote_undersampling["test_F1"])
f1_score_std = np.std(nested_scores_smote_undersampling["test_F1"])
print(f'F1 Score (max): {np.round(f1_score, 2)} +- {np.round(f1_score_std, 2)}')

f2_score = np.mean(nested_scores_smote_undersampling["test_F2"])
f2_score_std = np.std(nested_scores_smote_undersampling["test_F2"])
print(f'F2 Score (max): {np.round(f2_score, 2)} +- {np.round(f2_score_std, 2)}')

brier_score = -np.mean(nested_scores_smote_undersampling["test_Brier"])
brier_score_std = -np.std(nested_scores_smote_undersampling["test_Brier"])
print(f'Brier Score (min): {np.round(brier_score, 2)} +- {np.round(brier_score_std, 2)}')

# test_False_neg_scorer returns the number of test false negatives -> to get a % we need to divide by the number of test samples*100
false_neg_score = np.mean(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
false_neg_score_std = np.std(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
print(f'False negative: {int(np.round(false_neg_score, 0))}% +- {int(np.round(false_neg_score_std, 0))}')

false_pos_score = np.mean(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
false_pos_score_std = np.std(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
print(f'False positive: {int(np.round(false_pos_score, 0))}% +- {int(np.round(false_pos_score_std, 0))}')

HistGradientBoostingClassifier with hyperparameter gridsearch
AUC (max): 0.93 +- 0.05
F1 Score (max): 0.53 +- 0.1
F2 Score (max): 0.62 +- 0.12
Brier Score (min): 0.07 +- -0.02
False negative: 2% +- 1
False positive: 6% +- 3


#### Confidence intervals with test set bootstrap

In [12]:
best_params_list = []

for estimator in nested_scores_smote_undersampling["estimator"]:
    best_params_list.append(frozenset(estimator.best_params_.items()))

In [13]:
from collections import Counter
# Convert dict to a hashable tuple and count occurrences
most_common_params = Counter(best_params_list).most_common(1)[0][0]
final_params = dict(most_common_params)  # Convert back to dict


print("Final chosen hyperparameters:", final_params)

Final chosen hyperparameters: {'model__learning_rate': 0.01, 'under__sampling_strategy': 0.7, 'over__sampling_strategy': 0.2, 'over__k_neighbors': 5}


In [14]:
pipeline = Pipeline(steps=[('over', SMOTE(k_neighbors=5, sampling_strategy=0.2)), ('under', RandomUnderSampler(sampling_strategy=0.7)), ('model', HistGradientBoostingClassifier(learning_rate=0.01))])

In [15]:
y_pred = cross_val_predict(pipeline, X_features_imputed, y_outcome, cv=20, method='predict_proba')[:,1]

y_pred_binary = [1 if i>=0.5 else 0 for i in y_pred]

In [16]:
from sklearn.metrics import fbeta_score, confusion_matrix, roc_auc_score, f1_score, brier_score_loss

In [17]:
rng = np.random.RandomState(seed=12345)
idx = np.arange(len(y_outcome))

y_pred = np.asarray(y_pred)
y_pred_binary = np.asarray(y_pred_binary)
y = np.asarray(y_outcome)
X_dropped = np.asarray(X_features_imputed)

test_roc_auc = []
test_f1 = []
test_ftwos = []
test_brier = []
test_false_neg = []
test_false_pos = []

for i in range(200): # bootstrap with 200 rounds: random sampling with replacement of the predictions

    pred_idx = rng.choice(idx, size=len(idx), replace=True)
    
    roc_auc_test_boot = roc_auc_score(y_score=y_pred[pred_idx], y_true=y[pred_idx])
    f1_test_boot = f1_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx])
    f2_test_boot = fbeta_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx], beta=2)
    brier_test_boot = brier_score_loss(y_proba=y_pred[pred_idx], y_true=y[pred_idx])
    false_neg_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[1,0]
    false_pos_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[0,1]
    
    test_roc_auc.append(roc_auc_test_boot)
    test_f1.append(f1_test_boot)
    test_ftwos.append(f2_test_boot)
    test_brier.append(brier_test_boot)
    test_false_neg.append(false_neg_test_boot/len(idx)*100)
    test_false_pos.append(false_pos_test_boot/len(idx)*100)


In [18]:
print("Classification performance\n")

bootstrap_roc_auc_test_mean = np.mean(test_roc_auc)
ci_lower = np.percentile(test_roc_auc, 2.5)     # 2.5 percentile (alpha=0.025)
ci_upper = np.percentile(test_roc_auc, 97.5)
print(f"ROC AUC:         {bootstrap_roc_auc_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f1_test_mean = np.mean(test_f1)
ci_lower = np.percentile(test_f1, 2.5)
ci_upper = np.percentile(test_f1, 97.5)
print(f"F1:              {bootstrap_f1_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f2_test_mean = np.mean(test_ftwos)
ci_lower = np.percentile(test_ftwos, 2.5)
ci_upper = np.percentile(test_ftwos, 97.5)
print(f"F2:              {bootstrap_f2_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_brier_test_mean = np.mean(test_brier)
ci_lower = np.percentile(test_brier, 2.5)
ci_upper = np.percentile(test_brier, 97.5)
print(f"Brier loss:      {bootstrap_brier_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_neg_test_mean = np.mean(test_false_neg)
ci_lower = np.percentile(test_false_neg, 2.5)
ci_upper = np.percentile(test_false_neg, 97.5)
print(f"False negatives: {bootstrap_false_neg_test_mean:.2f}%;  95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_pos_test_mean = np.mean(test_false_pos)
ci_lower = np.percentile(test_false_pos, 2.5)
ci_upper = np.percentile(test_false_pos, 97.5)
print(f"False negatives: {bootstrap_false_pos_test_mean:.2f}%; 95% CI {ci_lower:.2f}-{ci_upper:.2f}")

Classification performance

ROC AUC:         0.92;   95% CI 0.87-0.97
F1:              0.55;   95% CI 0.42-0.66
F2:              0.64;   95% CI 0.51-0.75
Brier loss:      0.07;   95% CI 0.06-0.08
False negatives: 1.67%;  95% CI 0.75-2.81
False negatives: 5.40%; 95% CI 3.37-7.49


### Gradient boosting with prehospital + segmentation

In [ ]:
# run preprocessing outcome mortality

In [88]:
# Select columns 16 to 36
X_prehosp_and_segmentation = data_full.iloc[:, 15:35]

# Add segmentation columns
X_prehosp_and_segmentation = pd.concat([X_prehosp_and_segmentation, data_full.iloc[:, 101:115]], axis=1)

# Convert all columns to numeric, replacing non-numeric values and empty strings with NaN
X_prehosp_and_segmentation = X_prehosp_and_segmentation.apply(pd.to_numeric, errors='coerce')

# Include column name
X_prehosp_and_segmentation = pd.concat([X_prehosp_and_segmentation, data_full.iloc[:, 2]], axis=1)

In [89]:



# Verify if there are NaN values
print(f"Number of NaN values per column:\n{X_prehosp_and_segmentation.isna().sum()}")

# Display the resulting DataFrame
X_prehosp_and_segmentation.head()

Number of NaN values per column:
PAS  SMUR                    40
PAD  SMUR                    49
FC SMUR                      42
FR SMUR                     485
Shock Index SMUR             55
GCS SMUR                     17
GCS (M) SMUR                 30
Hémocue SMUR                243
Shock Index inversé          54
Shock index diastolique      62
Anomalie pupille SMUR        17
Fracas bassin                14
Amputation                   12
ACR SMUR                     11
Hémorragie ext SMUR          12
Ischémie                     12
Intubation prehosp           11
Expansion volémique          20
OsmoTH prehosp               11
Vasopresseur prehosp         12
supratentorial_IPH            0
supratentorial_SAH            0
supratentorial_Petechiae      0
supratentorial_Edema          0
infratentorial_IPH            0
infratentorial_SAH            0
infratentorial_Petechiae      0
infratentorial_Edema          0
brainstem_IPH                 0
brainstem_SAH                 0
brainst

,PAS SMUR,PAD SMUR,FC SMUR,FR SMUR,Shock Index SMUR,GCS SMUR,GCS (M) SMUR,Hémocue SMUR,Shock Index inversé,Shock index diastolique,...,infratentorial_SAH,infratentorial_Petechiae,infratentorial_Edema,brainstem_IPH,brainstem_SAH,brainstem_Petechiae,brainstem_Edema,SDH_y,EDH,name
0,190.0,103.0,137.0,NaN,0.72,15.0,6.0,NaN,1.39,1.33,...,0,0,0,0,0,0,0,2476,229,P0000
1,87.0,49.0,56.0,NaN,0.64,15.0,6.0,NaN,1.55,1.14,...,15,0,0,0,0,0,0,43,0,P0001
2,100.0,60.0,100.0,17.0,1.00,15.0,6.0,NaN,1.00,1.67,...,0,0,0,0,0,0,0,312,11685,P0002
3,101.0,64.0,120.0,NaN,1.19,14.0,6.0,13.1,0.84,1.88,...,0,0,0,0,0,0,0,11,0,P0003
4,110.0,71.0,107.0,18.0,0.97,15.0,6.0,15.8,1.03,1.51,...,0,0,0,0,0,0,0,796,0,P0004


In [90]:
X = X_prehosp_and_segmentation.copy()
y = get_mortality_6m()

    name  mortality
0  P0000          0
1  P0001          0
2  P0002          0
3  P0003          0
4  P0004          0
Outcome events : 33


/bettik/PROJECTS/pr-gin5_aini/fehrdelt/FastDiag/data_preprocessing_outcomes.py:236: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace({'1': 1, '0': 0, 'nd': np.nan, '8 Upper Good Recovery (Upper GR)': np.nan}).astype(float)
/bettik/PROJECTS/pr-gin5_aini/fehrdelt/FastDiag/data_preprocessing_outcomes.py:236: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace({'1': 1, '0': 0, 'nd': np.nan, '8 Upper Good Recovery (Upper GR)': np.nan}).astype(float)
/bettik/PROJECTS/pr-gin5_aini/fehrde

In [91]:
# Identify indexes where 'mortality' is NaN in y
nan_indexes = y.loc[pd.isna(y["mortality"]), :].index
print(f"Indexes with NaN in 'mortality': {nan_indexes}")

# Drop rows with NaN in y
y = y.drop(index=nan_indexes)

# Ensure 'name' is the index or used for alignment
X = X[~X['name'].isin(y.loc[nan_indexes, 'name'])]

# Verify the shapes after cleaning
print(f"Shape of y after cleaning: {y.shape}")
print(f"Shape of X after cleaning: {X.shape}")

# Vérifiez les valeurs uniques de 'name' dans y
print("Unique 'name' values in y:", y['name'].nunique())

# Vérifiez les valeurs uniques de 'name' dans X
print("Unique 'name' values in X:", X['name'].nunique())

# Noms dans y mais pas dans X
missing_in_X = set(y['name']) - set(X['name'])
print(f"Names in y but not in X: {missing_in_X}")

# Noms dans X mais pas dans y
missing_in_y = set(X['name']) - set(y['name'])
print(f"Names in X but not in y: {missing_in_y}")

# Trouver les noms communs
common_names = set(y['name']).intersection(set(X['name']))

# Filtrer y et X pour ne garder que les noms communs
y = y[y['name'].isin(common_names)]
X = X[X['name'].isin(common_names)]

# Vérifiez les dimensions après nettoyage
print(f"Shape of y after alignment: {y.shape}")
print(f"Shape of X after alignment: {X.shape}")

# Check for duplicates in 'name' in y
print("Number of duplicate names in y:", y['name'].duplicated().sum())

# Check for duplicates in 'name' in X
print("Number of duplicate names in X:", X['name'].duplicated().sum())

# Remove duplicates from both DataFrames
y = y.drop_duplicates(subset=['name'])
X = X.drop_duplicates(subset=['name'])

# Verify the shapes again
print(f"Shape of y after removing duplicates: {y.shape}")
print(f"Shape of X after removing duplicates: {X.shape}")

# Check if 'name' values are the same in both DataFrames
common_names = set(y['name']) == set(X['name'])
print(f"Are 'name' values aligned between y and X {common_names}")

# Drop the 'name' column from X_volumes_clinical_with_outcome
X_features = X.drop(columns=['name'])

# imputation with median 
X_features_imputed = X_features.fillna(X_features.median())

# Drop the 'name' column from y (if it exists)
y_outcome = y.drop(columns=['name'])

# Ensure y_outcome is a 1D array
y_outcome = y_outcome.values.ravel()  # Convert to 1D if using pandas DataFrame

FOLDS = 5
N_REPEATS = 3
nb_total_samples = len(y_outcome)

Indexes with NaN in 'mortality': Index([], dtype='int64')
Shape of y after cleaning: (566, 2)
Shape of X after cleaning: (534, 35)
Unique 'name' values in y: 562
Unique 'name' values in X: 534
Names in y but not in X: {'P0349', 'P0518', 'P0155', 'P0150', 'P0383', 'P0224', 'P0217', 'P0564', 'P0532', 'P0327', 'P0147', 'P0207', 'P0018', 'P0195', 'P0430', 'P0239', 'P0149', 'P0480', 'P0083', 'P0369', 'P0422', 'P0412', 'P0111', 'P0356', 'P0010', 'P0527', 'P0343', 'P0491'}
Names in X but not in y: set()
Shape of y after alignment: (536, 2)
Shape of X after alignment: (534, 35)
Number of duplicate names in y: 2
Number of duplicate names in X: 0
Shape of y after removing duplicates: (534, 2)
Shape of X after removing duplicates: (534, 35)
Are 'name' values aligned between y and X True


In [92]:
ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [93]:

pipeline_smote_under = Pipeline(steps=[('over', SMOTE()), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])
#pipeline_smote_under = Pipeline(steps=[('over', SMOTENC(categorical_features=["fracas_du_bassin", "amputation"])), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])


inner_cv = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=5, random_state=1)

p_grid = {"model__learning_rate": [0.01, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 1], "over__sampling_strategy": [0.1, 0.2, 0.3], "over__k_neighbors":[3,5,8], "under__sampling_strategy":[0.3, 0.5, 0.7]}
clf = GridSearchCV(estimator=pipeline_smote_under, param_grid=p_grid, scoring={'F2':ftwo_scorer}, refit='F2', cv=inner_cv)

outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

nested_scores_smote_undersampling = cross_validate(clf, X_features_imputed, y_outcome, 
                                                scoring={'F2':ftwo_scorer, 'ROC_AUC':'roc_auc', 'Recall':'recall_macro', 'F1':'f1', 'Brier':"neg_brier_score", 'False_neg_scorer':false_neg_scorer, 'False_pos_scorer':false_pos_scorer}, 
                                                cv=outer_cv, n_jobs=-1, return_estimator=True, return_indices=True)

print("HistGradientBoostingClassifier with hyperparameter gridsearch")

roc_auc_metric = np.mean(nested_scores_smote_undersampling["test_ROC_AUC"])
roc_auc_metric_std = np.std(nested_scores_smote_undersampling["test_ROC_AUC"])
print(f'AUC (max): {np.round(roc_auc_metric, 2)} +- {np.round(roc_auc_metric_std, 2)}')

f1_score = np.mean(nested_scores_smote_undersampling["test_F1"])
f1_score_std = np.std(nested_scores_smote_undersampling["test_F1"])
print(f'F1 Score (max): {np.round(f1_score, 2)} +- {np.round(f1_score_std, 2)}')

f2_score = np.mean(nested_scores_smote_undersampling["test_F2"])
f2_score_std = np.std(nested_scores_smote_undersampling["test_F2"])
print(f'F2 Score (max): {np.round(f2_score, 2)} +- {np.round(f2_score_std, 2)}')

brier_score = -np.mean(nested_scores_smote_undersampling["test_Brier"])
brier_score_std = -np.std(nested_scores_smote_undersampling["test_Brier"])
print(f'Brier Score (min): {np.round(brier_score, 2)} +- {np.round(brier_score_std, 2)}')

# test_False_neg_scorer returns the number of test false negatives -> to get a % we need to divide by the number of test samples*100
false_neg_score = np.mean(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
false_neg_score_std = np.std(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
print(f'False negative: {int(np.round(false_neg_score, 0))}% +- {int(np.round(false_neg_score_std, 0))}')

false_pos_score = np.mean(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
false_pos_score_std = np.std(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
print(f'False positive: {int(np.round(false_pos_score, 0))}% +- {int(np.round(false_pos_score_std, 0))}')

HistGradientBoostingClassifier with hyperparameter gridsearch
AUC (max): 0.94 +- 0.05
F1 Score (max): 0.68 +- 0.11
F2 Score (max): 0.72 +- 0.12
Brier Score (min): 0.04 +- -0.01
False negative: 2% +- 1
False positive: 3% +- 2


#### Confidence intervals with test set bootstrap

In [94]:
best_params_list = []

for estimator in nested_scores_smote_undersampling["estimator"]:
    best_params_list.append(frozenset(estimator.best_params_.items()))

In [95]:
from collections import Counter
# Convert dict to a hashable tuple and count occurrences
most_common_params = Counter(best_params_list).most_common(1)[0][0]
final_params = dict(most_common_params)  # Convert back to dict


print("Final chosen hyperparameters:", final_params)

Final chosen hyperparameters: {'model__learning_rate': 0.2, 'under__sampling_strategy': 0.7, 'over__k_neighbors': 8, 'over__sampling_strategy': 0.3}


In [96]:
pipeline = Pipeline(steps=[('over', SMOTE(k_neighbors=8, sampling_strategy=0.3)), ('under', RandomUnderSampler(sampling_strategy=0.7)), ('model', HistGradientBoostingClassifier(learning_rate=0.2))])

In [97]:
y_pred = cross_val_predict(pipeline, X_features_imputed, y_outcome, cv=20, method='predict_proba')[:,1]

y_pred_binary = [1 if i>=0.5 else 0 for i in y_pred]

In [98]:
from sklearn.metrics import fbeta_score, confusion_matrix, roc_auc_score, f1_score, brier_score_loss

In [99]:
rng = np.random.RandomState(seed=12345)
idx = np.arange(len(y_outcome))

y_pred = np.asarray(y_pred)
y_pred_binary = np.asarray(y_pred_binary)
y = np.asarray(y_outcome)
X_dropped = np.asarray(X_features_imputed)

test_roc_auc = []
test_f1 = []
test_ftwos = []
test_brier = []
test_false_neg = []
test_false_pos = []

for i in range(200): # bootstrap with 200 rounds: random sampling with replacement of the predictions

    pred_idx = rng.choice(idx, size=len(idx), replace=True)
    
    roc_auc_test_boot = roc_auc_score(y_score=y_pred[pred_idx], y_true=y[pred_idx])
    f1_test_boot = f1_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx])
    f2_test_boot = fbeta_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx], beta=2)
    brier_test_boot = brier_score_loss(y_proba=y_pred[pred_idx], y_true=y[pred_idx])
    false_neg_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[1,0]
    false_pos_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[0,1]
    
    test_roc_auc.append(roc_auc_test_boot)
    test_f1.append(f1_test_boot)
    test_ftwos.append(f2_test_boot)
    test_brier.append(brier_test_boot)
    test_false_neg.append(false_neg_test_boot/len(idx)*100)
    test_false_pos.append(false_pos_test_boot/len(idx)*100)


In [100]:
print("Classification performance\n")

bootstrap_roc_auc_test_mean = np.mean(test_roc_auc)
ci_lower = np.percentile(test_roc_auc, 2.5)     # 2.5 percentile (alpha=0.025)
ci_upper = np.percentile(test_roc_auc, 97.5)
print(f"ROC AUC:         {bootstrap_roc_auc_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f1_test_mean = np.mean(test_f1)
ci_lower = np.percentile(test_f1, 2.5)
ci_upper = np.percentile(test_f1, 97.5)
print(f"F1:              {bootstrap_f1_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f2_test_mean = np.mean(test_ftwos)
ci_lower = np.percentile(test_ftwos, 2.5)
ci_upper = np.percentile(test_ftwos, 97.5)
print(f"F2:              {bootstrap_f2_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_brier_test_mean = np.mean(test_brier)
ci_lower = np.percentile(test_brier, 2.5)
ci_upper = np.percentile(test_brier, 97.5)
print(f"Brier loss:      {bootstrap_brier_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_neg_test_mean = np.mean(test_false_neg)
ci_lower = np.percentile(test_false_neg, 2.5)
ci_upper = np.percentile(test_false_neg, 97.5)
print(f"False negatives: {bootstrap_false_neg_test_mean:.2f}%;  95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_pos_test_mean = np.mean(test_false_pos)
ci_lower = np.percentile(test_false_pos, 2.5)
ci_upper = np.percentile(test_false_pos, 97.5)
print(f"False negatives: {bootstrap_false_pos_test_mean:.2f}%; 95% CI {ci_lower:.2f}-{ci_upper:.2f}")

Classification performance

ROC AUC:         0.94;   95% CI 0.87-0.99
F1:              0.67;   95% CI 0.55-0.78
F2:              0.70;   95% CI 0.57-0.82
Brier loss:      0.04;   95% CI 0.02-0.05
False negatives: 1.65%;  95% CI 0.75-2.81
False negatives: 2.67%; 95% CI 1.31-3.94


### Gradient boosting with all DCA features

In [ ]:
# run preproc outcome

In [47]:
# Select columns
X_dca = data_full.iloc[:, 35:49]

# Convert all columns to numeric, replacing non-numeric values and empty strings with NaN
#X_dca = X_dca.apply(pd.to_numeric, errors='coerce')

# Display the resulting DataFrame
X_dca.head()




,PAS DCA,PAD DCA,FC DCA,Shock index DCA,Shock Index inversé.1,Shock index diastolique.1,GCS DCA,GCS (M) DCA,Température DCA,Hémocue DCA,Dextro DCA (mmol/l),DTC Vd,DTC IP,Osmothérapie
0,120,87,132,1.1,0.91,1.52,15,6,37.2,13.7,14.3,nd,nd,0
1,110,60,60,0.55,1.83,1,15,6,35.2,14,5.9,46,0.8,0
2,110,50,100,0.91,1.1,2,13,nd,38.2,12.7,6.4,55,1.3,0
3,120,90,120,1,1,1.33,15,6,37.8,12.8,6.8,32,0.9,0
4,120,60,110,0.92,1.09,1.83,15,6,37.1,15.6,9.4,nd,nd,0


In [48]:



# Verify if there are NaN values
print(f"Number of NaN values per column:\n{X_dca.isna().sum()}")

# Display the resulting DataFrame
X_dca.head()

Number of NaN values per column:
PAS DCA                       12
PAD DCA                       12
FC  DCA                       13
Shock index DCA                0
Shock Index inversé.1          0
Shock index diastolique.1      0
GCS DCA                       14
GCS (M) DCA                   20
Température DCA               20
Hémocue DCA                   25
Dextro DCA (mmol/l)           53
DTC Vd                        16
DTC IP                       329
Osmothérapie                  11
dtype: int64


,PAS DCA,PAD DCA,FC DCA,Shock index DCA,Shock Index inversé.1,Shock index diastolique.1,GCS DCA,GCS (M) DCA,Température DCA,Hémocue DCA,Dextro DCA (mmol/l),DTC Vd,DTC IP,Osmothérapie
0,120,87,132,1.1,0.91,1.52,15,6,37.2,13.7,14.3,nd,nd,0
1,110,60,60,0.55,1.83,1,15,6,35.2,14,5.9,46,0.8,0
2,110,50,100,0.91,1.1,2,13,nd,38.2,12.7,6.4,55,1.3,0
3,120,90,120,1,1,1.33,15,6,37.8,12.8,6.8,32,0.9,0
4,120,60,110,0.92,1.09,1.83,15,6,37.1,15.6,9.4,nd,nd,0


In [49]:
import numpy as np

# Define the conditions for dtc
def calculate_dtc(row):
    # Condition for dtc = NA
    if ('nr' in [row['DTC Vd'], row['DTC IP']] or
        'non réalisé' in [row['DTC Vd'], row['DTC IP']]):
        return np.nan
    # Condition for dtc = 1
    if (row['DTC Vd'] == 'Pathologique' or row['DTC IP'] == 'Pathologique' or
        (is_numeric(row['DTC Vd']) and float(row['DTC Vd']) < 30) or
        (is_numeric(row['DTC IP']) and float(row['DTC IP']) > 1.2)):
        return 1
    # Default case for dtc = 0
    return 0

# Helper function to check if a value is numeric
def is_numeric(value):
    try:
        float(value)
        return True
    except (ValueError, TypeError):
        return False

# Apply the function row-wise to create the new column
X_dca['dtc'] = X_dca.apply(calculate_dtc, axis=1)

# Display the resulting DataFrame
X_dca.head()

,PAS DCA,PAD DCA,FC DCA,Shock index DCA,Shock Index inversé.1,Shock index diastolique.1,GCS DCA,GCS (M) DCA,Température DCA,Hémocue DCA,Dextro DCA (mmol/l),DTC Vd,DTC IP,Osmothérapie,dtc
0,120,87,132,1.1,0.91,1.52,15,6,37.2,13.7,14.3,nd,nd,0,0.0
1,110,60,60,0.55,1.83,1,15,6,35.2,14,5.9,46,0.8,0,0.0
2,110,50,100,0.91,1.1,2,13,nd,38.2,12.7,6.4,55,1.3,0,1.0
3,120,90,120,1,1,1.33,15,6,37.8,12.8,6.8,32,0.9,0,0.0
4,120,60,110,0.92,1.09,1.83,15,6,37.1,15.6,9.4,nd,nd,0,0.0


In [50]:
# Convert all columns to numeric, replacing non-numeric values and empty strings with NaN
X_dca = X_dca.apply(pd.to_numeric, errors='coerce')

# Display the resulting DataFrame
X_dca.head()

# Verify if there are NaN values
print(f"Number of NaN values per column:\n{X_dca.isna().sum()}")

Number of NaN values per column:
PAS DCA                       14
PAD DCA                       14
FC  DCA                       17
Shock index DCA               20
Shock Index inversé.1         20
Shock index diastolique.1     20
GCS DCA                       30
GCS (M) DCA                   33
Température DCA              402
Hémocue DCA                   37
Dextro DCA (mmol/l)           75
DTC Vd                       453
DTC IP                       452
Osmothérapie                 395
dtc                          163
dtype: int64


In [51]:
# Include column name
X_dca = pd.concat([X_dca, data_full.iloc[:, 2]], axis=1)
X = X_dca.copy()
y = get_mortality_6m()

    name  mortality
0  P0000          0
1  P0001          0
2  P0002          0
3  P0003          0
4  P0004          0
Outcome events : 33


/bettik/PROJECTS/pr-gin5_aini/fehrdelt/FastDiag/data_preprocessing_outcomes.py:236: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace({'1': 1, '0': 0, 'nd': np.nan, '8 Upper Good Recovery (Upper GR)': np.nan}).astype(float)
/bettik/PROJECTS/pr-gin5_aini/fehrdelt/FastDiag/data_preprocessing_outcomes.py:236: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace({'1': 1, '0': 0, 'nd': np.nan, '8 Upper Good Recovery (Upper GR)': np.nan}).astype(float)
/bettik/PROJECTS/pr-gin5_aini/fehrde

In [53]:
# Identify indexes where 'mortality' is NaN in y
nan_indexes = y.loc[pd.isna(y["mortality"]), :].index
print(f"Indexes with NaN in 'mortality': {nan_indexes}")

# Drop rows with NaN in y
y = y.drop(index=nan_indexes)

# Ensure 'name' is the index or used for alignment
X = X[~X['name'].isin(y.loc[nan_indexes, 'name'])]

# Verify the shapes after cleaning
print(f"Shape of y after cleaning: {y.shape}")
print(f"Shape of X after cleaning: {X.shape}")

# Vérifiez les valeurs uniques de 'name' dans y
print("Unique 'name' values in y:", y['name'].nunique())

# Vérifiez les valeurs uniques de 'name' dans X
print("Unique 'name' values in X:", X['name'].nunique())

# Noms dans y mais pas dans X
missing_in_X = set(y['name']) - set(X['name'])
print(f"Names in y but not in X: {missing_in_X}")

# Noms dans X mais pas dans y
missing_in_y = set(X['name']) - set(y['name'])
print(f"Names in X but not in y: {missing_in_y}")

# Trouver les noms communs
common_names = set(y['name']).intersection(set(X['name']))

# Filtrer y et X pour ne garder que les noms communs
y = y[y['name'].isin(common_names)]
X = X[X['name'].isin(common_names)]

# Vérifiez les dimensions après nettoyage
print(f"Shape of y after alignment: {y.shape}")
print(f"Shape of X after alignment: {X.shape}")

# Check for duplicates in 'name' in y
print("Number of duplicate names in y:", y['name'].duplicated().sum())

# Check for duplicates in 'name' in X
print("Number of duplicate names in X:", X['name'].duplicated().sum())

# Remove duplicates from both DataFrames
y = y.drop_duplicates(subset=['name'])
X = X.drop_duplicates(subset=['name'])

# Verify the shapes again
print(f"Shape of y after removing duplicates: {y.shape}")
print(f"Shape of X after removing duplicates: {X.shape}")

# Check if 'name' values are the same in both DataFrames
common_names = set(y['name']) == set(X['name'])
print(f"Are 'name' values aligned between y and X {common_names}")

# Trier y et X par la colonne 'name'
y = y.sort_values(by='name').reset_index(drop=True)
X = X.sort_values(by='name').reset_index(drop=True)

# Vérifier si les noms sont alignés
print((y['name'].values == X['name'].values).all())

# Drop the 'name' column from X
#X_features = X.drop(columns=['name', 'mortalité J7'])
X_features = X.drop(columns=['name'])

# imputation with median 
X_features_imputed = X_features.fillna(X_features.median())

# Drop the 'name' column from y (if it exists)
y_outcome = y.drop(columns=['name'])

# Ensure y_outcome is a 1D array
y_outcome = y_outcome.values.ravel()  # Convert to 1D if using pandas DataFrame

FOLDS = 5
N_REPEATS = 3
nb_total_samples = len(y_outcome)

Indexes with NaN in 'mortality': Index([], dtype='int64')
Shape of y after cleaning: (534, 2)
Shape of X after cleaning: (534, 16)
Unique 'name' values in y: 534
Unique 'name' values in X: 534
Names in y but not in X: set()
Names in X but not in y: set()
Shape of y after alignment: (534, 2)
Shape of X after alignment: (534, 16)
Number of duplicate names in y: 0
Number of duplicate names in X: 0
Shape of y after removing duplicates: (534, 2)
Shape of X after removing duplicates: (534, 16)
Are 'name' values aligned between y and X True
True


In [54]:
X_features.head()

,PAS DCA,PAD DCA,FC DCA,Shock index DCA,Shock Index inversé.1,Shock index diastolique.1,GCS DCA,GCS (M) DCA,Température DCA,Hémocue DCA,Dextro DCA (mmol/l),DTC Vd,DTC IP,Osmothérapie,dtc
0,120.0,87.0,132.0,1.10,0.91,1.52,15.0,6.0,37.2,13.7,14.3,NaN,NaN,0.0,0.0
1,110.0,60.0,60.0,0.55,1.83,1.00,15.0,6.0,35.2,14.0,5.9,46.0,0.8,0.0,0.0
2,110.0,50.0,100.0,0.91,1.10,2.00,13.0,NaN,38.2,12.7,6.4,55.0,1.3,0.0,1.0
3,120.0,90.0,120.0,1.00,1.00,1.33,15.0,6.0,37.8,12.8,6.8,32.0,0.9,0.0,0.0
4,120.0,60.0,110.0,0.92,1.09,1.83,15.0,6.0,37.1,15.6,9.4,NaN,NaN,0.0,0.0


In [55]:

pipeline_smote_under = Pipeline(steps=[('over', SMOTE()), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])
#pipeline_smote_under = Pipeline(steps=[('over', SMOTENC(categorical_features=["fracas_du_bassin", "amputation"])), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])


inner_cv = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=5, random_state=1)

p_grid = {"model__learning_rate": [0.01, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 1], "over__sampling_strategy": [0.1, 0.2, 0.3], "over__k_neighbors":[3,5,8], "under__sampling_strategy":[0.3, 0.5, 0.7]}
clf = GridSearchCV(estimator=pipeline_smote_under, param_grid=p_grid, scoring={'F2':ftwo_scorer}, refit='F2', cv=inner_cv)

outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

nested_scores_smote_undersampling = cross_validate(clf, X_features_imputed, y_outcome, 
                                                scoring={'F2':ftwo_scorer, 'ROC_AUC':'roc_auc', 'Recall':'recall_macro', 'F1':'f1', 'Brier':"neg_brier_score", 'False_neg_scorer':false_neg_scorer, 'False_pos_scorer':false_pos_scorer}, 
                                                cv=outer_cv, n_jobs=-1, return_estimator=True, return_indices=True)

print("HistGradientBoostingClassifier with hyperparameter gridsearch")

roc_auc_metric = np.mean(nested_scores_smote_undersampling["test_ROC_AUC"])
roc_auc_metric_std = np.std(nested_scores_smote_undersampling["test_ROC_AUC"])
print(f'AUC (max): {np.round(roc_auc_metric, 2)} +- {np.round(roc_auc_metric_std, 2)}')

f1_score = np.mean(nested_scores_smote_undersampling["test_F1"])
f1_score_std = np.std(nested_scores_smote_undersampling["test_F1"])
print(f'F1 Score (max): {np.round(f1_score, 2)} +- {np.round(f1_score_std, 2)}')

f2_score = np.mean(nested_scores_smote_undersampling["test_F2"])
f2_score_std = np.std(nested_scores_smote_undersampling["test_F2"])
print(f'F2 Score (max): {np.round(f2_score, 2)} +- {np.round(f2_score_std, 2)}')

brier_score = -np.mean(nested_scores_smote_undersampling["test_Brier"])
brier_score_std = -np.std(nested_scores_smote_undersampling["test_Brier"])
print(f'Brier Score (min): {np.round(brier_score, 2)} +- {np.round(brier_score_std, 2)}')

# test_False_neg_scorer returns the number of test false negatives -> to get a % we need to divide by the number of test samples*100
false_neg_score = np.mean(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
false_neg_score_std = np.std(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
print(f'False negative: {int(np.round(false_neg_score, 0))}% +- {int(np.round(false_neg_score_std, 0))}')

false_pos_score = np.mean(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
false_pos_score_std = np.std(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
print(f'False positive: {int(np.round(false_pos_score, 0))}% +- {int(np.round(false_pos_score_std, 0))}')

HistGradientBoostingClassifier with hyperparameter gridsearch
AUC (max): 0.88 +- 0.07
F1 Score (max): 0.45 +- 0.1
F2 Score (max): 0.56 +- 0.12
Brier Score (min): 0.09 +- -0.01
False negative: 2% +- 1
False positive: 9% +- 3


#### Confidence intervals with test set bootstrap

In [56]:
best_params_list = []

for estimator in nested_scores_smote_undersampling["estimator"]:
    best_params_list.append(frozenset(estimator.best_params_.items()))

In [57]:
from collections import Counter
# Convert dict to a hashable tuple and count occurrences
most_common_params = Counter(best_params_list).most_common(1)[0][0]
final_params = dict(most_common_params)  # Convert back to dict


print("Final chosen hyperparameters:", final_params)

Final chosen hyperparameters: {'model__learning_rate': 0.01, 'under__sampling_strategy': 0.7, 'over__k_neighbors': 3, 'over__sampling_strategy': 0.2}


In [58]:
pipeline = Pipeline(steps=[('over', SMOTE(k_neighbors=3, sampling_strategy=0.2)), ('under', RandomUnderSampler(sampling_strategy=0.7)), ('model', HistGradientBoostingClassifier(learning_rate=0.01))])

In [59]:
y_pred = cross_val_predict(pipeline, X_features_imputed, y_outcome, cv=20, method='predict_proba')[:,1]

y_pred_binary = [1 if i>=0.5 else 0 for i in y_pred]

In [60]:
from sklearn.metrics import fbeta_score, confusion_matrix, roc_auc_score, f1_score, brier_score_loss

In [61]:
rng = np.random.RandomState(seed=12345)
idx = np.arange(len(y_outcome))

y_pred = np.asarray(y_pred)
y_pred_binary = np.asarray(y_pred_binary)
y = np.asarray(y_outcome)
X_dropped = np.asarray(X_features_imputed)

test_roc_auc = []
test_f1 = []
test_ftwos = []
test_brier = []
test_false_neg = []
test_false_pos = []

for i in range(200): # bootstrap with 200 rounds: random sampling with replacement of the predictions

    pred_idx = rng.choice(idx, size=len(idx), replace=True)
    
    roc_auc_test_boot = roc_auc_score(y_score=y_pred[pred_idx], y_true=y[pred_idx])
    f1_test_boot = f1_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx])
    f2_test_boot = fbeta_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx], beta=2)
    brier_test_boot = brier_score_loss(y_proba=y_pred[pred_idx], y_true=y[pred_idx])
    false_neg_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[1,0]
    false_pos_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[0,1]
    
    test_roc_auc.append(roc_auc_test_boot)
    test_f1.append(f1_test_boot)
    test_ftwos.append(f2_test_boot)
    test_brier.append(brier_test_boot)
    test_false_neg.append(false_neg_test_boot/len(idx)*100)
    test_false_pos.append(false_pos_test_boot/len(idx)*100)


In [62]:
print("Classification performance\n")

bootstrap_roc_auc_test_mean = np.mean(test_roc_auc)
ci_lower = np.percentile(test_roc_auc, 2.5)     # 2.5 percentile (alpha=0.025)
ci_upper = np.percentile(test_roc_auc, 97.5)
print(f"ROC AUC:         {bootstrap_roc_auc_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f1_test_mean = np.mean(test_f1)
ci_lower = np.percentile(test_f1, 2.5)
ci_upper = np.percentile(test_f1, 97.5)
print(f"F1:              {bootstrap_f1_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f2_test_mean = np.mean(test_ftwos)
ci_lower = np.percentile(test_ftwos, 2.5)
ci_upper = np.percentile(test_ftwos, 97.5)
print(f"F2:              {bootstrap_f2_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_brier_test_mean = np.mean(test_brier)
ci_lower = np.percentile(test_brier, 2.5)
ci_upper = np.percentile(test_brier, 97.5)
print(f"Brier loss:      {bootstrap_brier_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_neg_test_mean = np.mean(test_false_neg)
ci_lower = np.percentile(test_false_neg, 2.5)
ci_upper = np.percentile(test_false_neg, 97.5)
print(f"False negatives: {bootstrap_false_neg_test_mean:.2f}%;  95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_pos_test_mean = np.mean(test_false_pos)
ci_lower = np.percentile(test_false_pos, 2.5)
ci_upper = np.percentile(test_false_pos, 97.5)
print(f"False negatives: {bootstrap_false_pos_test_mean:.2f}%; 95% CI {ci_lower:.2f}-{ci_upper:.2f}")

Classification performance

ROC AUC:         0.87;   95% CI 0.81-0.94
F1:              0.46;   95% CI 0.33-0.57
F2:              0.58;   95% CI 0.44-0.69
Brier loss:      0.09;   95% CI 0.08-0.10
False negatives: 1.84%;  95% CI 0.74-3.18
False negatives: 8.16%; 95% CI 5.99-10.49


### Gradient boosting with all DCA + segmentation

In [63]:
# run preproc outcome

In [72]:
# Select columns (DCA)
X_dca_and_segmentation = data_full.iloc[:, 35:49] # add 92:

# Add segmentation columns
X_dca_and_segmentation = pd.concat([X_dca_and_segmentation, data_full.iloc[:, 101:115]], axis=1)


# Convert all columns to numeric, replacing non-numeric values and empty strings with NaN
#X_dca = X_dca.apply(pd.to_numeric, errors='coerce')

# Display the resulting DataFrame
X_dca_and_segmentation.head()




,PAS DCA,PAD DCA,FC DCA,Shock index DCA,Shock Index inversé.1,Shock index diastolique.1,GCS DCA,GCS (M) DCA,Température DCA,Hémocue DCA,...,infratentorial_IPH,infratentorial_SAH,infratentorial_Petechiae,infratentorial_Edema,brainstem_IPH,brainstem_SAH,brainstem_Petechiae,brainstem_Edema,SDH_y,EDH
0,120,87,132,1.1,0.91,1.52,15,6,37.2,13.7,...,0,0,0,0,0,0,0,0,2476,229
1,110,60,60,0.55,1.83,1,15,6,35.2,14,...,0,15,0,0,0,0,0,0,43,0
2,110,50,100,0.91,1.1,2,13,nd,38.2,12.7,...,0,0,0,0,0,0,0,0,312,11685
3,120,90,120,1,1,1.33,15,6,37.8,12.8,...,0,0,0,0,0,0,0,0,11,0
4,120,60,110,0.92,1.09,1.83,15,6,37.1,15.6,...,0,0,0,0,0,0,0,0,796,0


In [74]:



# Verify if there are NaN values
print(f"Number of NaN values per column:\n{X_dca_and_segmentation.isna().sum()}")

# Display the resulting DataFrame
X_dca_and_segmentation.head()

Number of NaN values per column:
PAS DCA                       12
PAD DCA                       12
FC  DCA                       13
Shock index DCA                0
Shock Index inversé.1          0
Shock index diastolique.1      0
GCS DCA                       14
GCS (M) DCA                   20
Température DCA               20
Hémocue DCA                   25
Dextro DCA (mmol/l)           53
DTC Vd                        16
DTC IP                       329
Osmothérapie                  11
supratentorial_IPH             0
supratentorial_SAH             0
supratentorial_Petechiae       0
supratentorial_Edema           0
infratentorial_IPH             0
infratentorial_SAH             0
infratentorial_Petechiae       0
infratentorial_Edema           0
brainstem_IPH                  0
brainstem_SAH                  0
brainstem_Petechiae            0
brainstem_Edema                0
SDH_y                          0
EDH                            0
dtype: int64


,PAS DCA,PAD DCA,FC DCA,Shock index DCA,Shock Index inversé.1,Shock index diastolique.1,GCS DCA,GCS (M) DCA,Température DCA,Hémocue DCA,...,infratentorial_IPH,infratentorial_SAH,infratentorial_Petechiae,infratentorial_Edema,brainstem_IPH,brainstem_SAH,brainstem_Petechiae,brainstem_Edema,SDH_y,EDH
0,120,87,132,1.1,0.91,1.52,15,6,37.2,13.7,...,0,0,0,0,0,0,0,0,2476,229
1,110,60,60,0.55,1.83,1,15,6,35.2,14,...,0,15,0,0,0,0,0,0,43,0
2,110,50,100,0.91,1.1,2,13,nd,38.2,12.7,...,0,0,0,0,0,0,0,0,312,11685
3,120,90,120,1,1,1.33,15,6,37.8,12.8,...,0,0,0,0,0,0,0,0,11,0
4,120,60,110,0.92,1.09,1.83,15,6,37.1,15.6,...,0,0,0,0,0,0,0,0,796,0


In [75]:
import numpy as np

# Define the conditions for dtc
def calculate_dtc(row):
    # Condition for dtc = NA
    if ('nr' in [row['DTC Vd'], row['DTC IP']] or
        'non réalisé' in [row['DTC Vd'], row['DTC IP']]):
        return np.nan
    # Condition for dtc = 1
    if (row['DTC Vd'] == 'Pathologique' or row['DTC IP'] == 'Pathologique' or
        (is_numeric(row['DTC Vd']) and float(row['DTC Vd']) < 30) or
        (is_numeric(row['DTC IP']) and float(row['DTC IP']) > 1.2)):
        return 1
    # Default case for dtc = 0
    return 0

# Helper function to check if a value is numeric
def is_numeric(value):
    try:
        float(value)
        return True
    except (ValueError, TypeError):
        return False

# Apply the function row-wise to create the new column
X_dca_and_segmentation['dtc'] = X_dca_and_segmentation.apply(calculate_dtc, axis=1)

# Display the resulting DataFrame
X_dca_and_segmentation.head()

,PAS DCA,PAD DCA,FC DCA,Shock index DCA,Shock Index inversé.1,Shock index diastolique.1,GCS DCA,GCS (M) DCA,Température DCA,Hémocue DCA,...,infratentorial_SAH,infratentorial_Petechiae,infratentorial_Edema,brainstem_IPH,brainstem_SAH,brainstem_Petechiae,brainstem_Edema,SDH_y,EDH,dtc
0,120,87,132,1.1,0.91,1.52,15,6,37.2,13.7,...,0,0,0,0,0,0,0,2476,229,0.0
1,110,60,60,0.55,1.83,1,15,6,35.2,14,...,15,0,0,0,0,0,0,43,0,0.0
2,110,50,100,0.91,1.1,2,13,nd,38.2,12.7,...,0,0,0,0,0,0,0,312,11685,1.0
3,120,90,120,1,1,1.33,15,6,37.8,12.8,...,0,0,0,0,0,0,0,11,0,0.0
4,120,60,110,0.92,1.09,1.83,15,6,37.1,15.6,...,0,0,0,0,0,0,0,796,0,0.0


In [76]:
# Convert all columns to numeric, replacing non-numeric values and empty strings with NaN
X_dca_and_segmentation = X_dca_and_segmentation.apply(pd.to_numeric, errors='coerce')

# Display the resulting DataFrame
X_dca_and_segmentation.head()

# Verify if there are NaN values
print(f"Number of NaN values per column:\n{X_dca_and_segmentation.isna().sum()}")

Number of NaN values per column:
PAS DCA                       14
PAD DCA                       14
FC  DCA                       17
Shock index DCA               20
Shock Index inversé.1         20
Shock index diastolique.1     20
GCS DCA                       30
GCS (M) DCA                   33
Température DCA              402
Hémocue DCA                   37
Dextro DCA (mmol/l)           75
DTC Vd                       453
DTC IP                       452
Osmothérapie                 395
supratentorial_IPH             0
supratentorial_SAH             0
supratentorial_Petechiae       0
supratentorial_Edema           0
infratentorial_IPH             0
infratentorial_SAH             0
infratentorial_Petechiae       0
infratentorial_Edema           0
brainstem_IPH                  0
brainstem_SAH                  0
brainstem_Petechiae            0
brainstem_Edema                0
SDH_y                          0
EDH                            0
dtc                          163
dtype: int

In [77]:
# Include column name
X_dca_and_segmentation = pd.concat([X_dca_and_segmentation, data_full.iloc[:, 2]], axis=1)
X = X_dca_and_segmentation.copy()
y = get_mortality_6m()

    name  mortality
0  P0000          0
1  P0001          0
2  P0002          0
3  P0003          0
4  P0004          0
Outcome events : 33


/bettik/PROJECTS/pr-gin5_aini/fehrdelt/FastDiag/data_preprocessing_outcomes.py:236: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace({'1': 1, '0': 0, 'nd': np.nan, '8 Upper Good Recovery (Upper GR)': np.nan}).astype(float)
/bettik/PROJECTS/pr-gin5_aini/fehrdelt/FastDiag/data_preprocessing_outcomes.py:236: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace({'1': 1, '0': 0, 'nd': np.nan, '8 Upper Good Recovery (Upper GR)': np.nan}).astype(float)
/bettik/PROJECTS/pr-gin5_aini/fehrde

In [78]:
# Identify indexes where 'mortality' is NaN in y
nan_indexes = y.loc[pd.isna(y["mortality"]), :].index
print(f"Indexes with NaN in 'mortality': {nan_indexes}")

# Drop rows with NaN in y
y = y.drop(index=nan_indexes)

# Ensure 'name' is the index or used for alignment
X = X[~X['name'].isin(y.loc[nan_indexes, 'name'])]

# Verify the shapes after cleaning
print(f"Shape of y after cleaning: {y.shape}")
print(f"Shape of X after cleaning: {X.shape}")

# Vérifiez les valeurs uniques de 'name' dans y
print("Unique 'name' values in y:", y['name'].nunique())

# Vérifiez les valeurs uniques de 'name' dans X
print("Unique 'name' values in X:", X['name'].nunique())

# Noms dans y mais pas dans X
missing_in_X = set(y['name']) - set(X['name'])
print(f"Names in y but not in X: {missing_in_X}")

# Noms dans X mais pas dans y
missing_in_y = set(X['name']) - set(y['name'])
print(f"Names in X but not in y: {missing_in_y}")

# Trouver les noms communs
common_names = set(y['name']).intersection(set(X['name']))

# Filtrer y et X pour ne garder que les noms communs
y = y[y['name'].isin(common_names)]
X = X[X['name'].isin(common_names)]

# Vérifiez les dimensions après nettoyage
print(f"Shape of y after alignment: {y.shape}")
print(f"Shape of X after alignment: {X.shape}")

# Check for duplicates in 'name' in y
print("Number of duplicate names in y:", y['name'].duplicated().sum())

# Check for duplicates in 'name' in X
print("Number of duplicate names in X:", X['name'].duplicated().sum())

# Remove duplicates from both DataFrames
y = y.drop_duplicates(subset=['name'])
X = X.drop_duplicates(subset=['name'])

# Verify the shapes again
print(f"Shape of y after removing duplicates: {y.shape}")
print(f"Shape of X after removing duplicates: {X.shape}")

# Check if 'name' values are the same in both DataFrames
common_names = set(y['name']) == set(X['name'])
print(f"Are 'name' values aligned between y and X {common_names}")

# Trier y et X par la colonne 'name'
y = y.sort_values(by='name').reset_index(drop=True)
X = X.sort_values(by='name').reset_index(drop=True)

# Vérifier si les noms sont alignés
print((y['name'].values == X['name'].values).all())

# Drop the 'name' column from X
#X_features = X.drop(columns=['name', 'mortalité J7'])
X_features = X.drop(columns=['name'])

# imputation with median 
X_features_imputed = X_features.fillna(X_features.median())

# Drop the 'name' column from y (if it exists)
y_outcome = y.drop(columns=['name'])

# Ensure y_outcome is a 1D array
y_outcome = y_outcome.values.ravel()  # Convert to 1D if using pandas DataFrame

FOLDS = 5
N_REPEATS = 3
nb_total_samples = len(y_outcome)

Indexes with NaN in 'mortality': Index([], dtype='int64')
Shape of y after cleaning: (566, 2)
Shape of X after cleaning: (534, 30)
Unique 'name' values in y: 562
Unique 'name' values in X: 534
Names in y but not in X: {'P0349', 'P0518', 'P0155', 'P0150', 'P0383', 'P0224', 'P0217', 'P0564', 'P0532', 'P0327', 'P0147', 'P0207', 'P0018', 'P0195', 'P0430', 'P0239', 'P0149', 'P0480', 'P0083', 'P0369', 'P0422', 'P0412', 'P0111', 'P0356', 'P0010', 'P0527', 'P0343', 'P0491'}
Names in X but not in y: set()
Shape of y after alignment: (536, 2)
Shape of X after alignment: (534, 30)
Number of duplicate names in y: 2
Number of duplicate names in X: 0
Shape of y after removing duplicates: (534, 2)
Shape of X after removing duplicates: (534, 30)
Are 'name' values aligned between y and X True
True


In [79]:
X_features.head()

,PAS DCA,PAD DCA,FC DCA,Shock index DCA,Shock Index inversé.1,Shock index diastolique.1,GCS DCA,GCS (M) DCA,Température DCA,Hémocue DCA,...,infratentorial_SAH,infratentorial_Petechiae,infratentorial_Edema,brainstem_IPH,brainstem_SAH,brainstem_Petechiae,brainstem_Edema,SDH_y,EDH,dtc
0,120.0,87.0,132.0,1.10,0.91,1.52,15.0,6.0,37.2,13.7,...,0,0,0,0,0,0,0,2476,229,0.0
1,110.0,60.0,60.0,0.55,1.83,1.00,15.0,6.0,35.2,14.0,...,15,0,0,0,0,0,0,43,0,0.0
2,110.0,50.0,100.0,0.91,1.10,2.00,13.0,NaN,38.2,12.7,...,0,0,0,0,0,0,0,312,11685,1.0
3,120.0,90.0,120.0,1.00,1.00,1.33,15.0,6.0,37.8,12.8,...,0,0,0,0,0,0,0,11,0,0.0
4,120.0,60.0,110.0,0.92,1.09,1.83,15.0,6.0,37.1,15.6,...,0,0,0,0,0,0,0,796,0,0.0


In [80]:

pipeline_smote_under = Pipeline(steps=[('over', SMOTE()), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])
#pipeline_smote_under = Pipeline(steps=[('over', SMOTENC(categorical_features=["fracas_du_bassin", "amputation"])), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])


inner_cv = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=5, random_state=1)

p_grid = {"model__learning_rate": [0.01, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 1], "over__sampling_strategy": [0.1, 0.2, 0.3], "over__k_neighbors":[3,5,8], "under__sampling_strategy":[0.3, 0.5, 0.7]}
clf = GridSearchCV(estimator=pipeline_smote_under, param_grid=p_grid, scoring={'F2':ftwo_scorer}, refit='F2', cv=inner_cv)

outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

nested_scores_smote_undersampling = cross_validate(clf, X_features_imputed, y_outcome, 
                                                scoring={'F2':ftwo_scorer, 'ROC_AUC':'roc_auc', 'Recall':'recall_macro', 'F1':'f1', 'Brier':"neg_brier_score", 'False_neg_scorer':false_neg_scorer, 'False_pos_scorer':false_pos_scorer}, 
                                                cv=outer_cv, n_jobs=-1, return_estimator=True, return_indices=True)

print("HistGradientBoostingClassifier with hyperparameter gridsearch")

roc_auc_metric = np.mean(nested_scores_smote_undersampling["test_ROC_AUC"])
roc_auc_metric_std = np.std(nested_scores_smote_undersampling["test_ROC_AUC"])
print(f'AUC (max): {np.round(roc_auc_metric, 2)} +- {np.round(roc_auc_metric_std, 2)}')

f1_score = np.mean(nested_scores_smote_undersampling["test_F1"])
f1_score_std = np.std(nested_scores_smote_undersampling["test_F1"])
print(f'F1 Score (max): {np.round(f1_score, 2)} +- {np.round(f1_score_std, 2)}')

f2_score = np.mean(nested_scores_smote_undersampling["test_F2"])
f2_score_std = np.std(nested_scores_smote_undersampling["test_F2"])
print(f'F2 Score (max): {np.round(f2_score, 2)} +- {np.round(f2_score_std, 2)}')

brier_score = -np.mean(nested_scores_smote_undersampling["test_Brier"])
brier_score_std = -np.std(nested_scores_smote_undersampling["test_Brier"])
print(f'Brier Score (min): {np.round(brier_score, 2)} +- {np.round(brier_score_std, 2)}')

# test_False_neg_scorer returns the number of test false negatives -> to get a % we need to divide by the number of test samples*100
false_neg_score = np.mean(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
false_neg_score_std = np.std(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
print(f'False negative: {int(np.round(false_neg_score, 0))}% +- {int(np.round(false_neg_score_std, 0))}')

false_pos_score = np.mean(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
false_pos_score_std = np.std(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
print(f'False positive: {int(np.round(false_pos_score, 0))}% +- {int(np.round(false_pos_score_std, 0))}')

HistGradientBoostingClassifier with hyperparameter gridsearch
AUC (max): 0.89 +- 0.06
F1 Score (max): 0.46 +- 0.11
F2 Score (max): 0.53 +- 0.14
Brier Score (min): 0.07 +- -0.02
False negative: 2% +- 1
False positive: 7% +- 3


#### Confidence intervals with test set bootstrap

In [81]:
best_params_list = []

for estimator in nested_scores_smote_undersampling["estimator"]:
    best_params_list.append(frozenset(estimator.best_params_.items()))

In [82]:
from collections import Counter
# Convert dict to a hashable tuple and count occurrences
most_common_params = Counter(best_params_list).most_common(1)[0][0]
final_params = dict(most_common_params)  # Convert back to dict


print("Final chosen hyperparameters:", final_params)

Final chosen hyperparameters: {'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.7, 'model__learning_rate': 1, 'over__k_neighbors': 5}


In [83]:
pipeline = Pipeline(steps=[('over', SMOTE(k_neighbors=5, sampling_strategy=0.2)), ('under', RandomUnderSampler(sampling_strategy=0.7)), ('model', HistGradientBoostingClassifier(learning_rate=1))])

In [84]:
y_pred = cross_val_predict(pipeline, X_features_imputed, y_outcome, cv=20, method='predict_proba')[:,1]

y_pred_binary = [1 if i>=0.5 else 0 for i in y_pred]

In [85]:
from sklearn.metrics import fbeta_score, confusion_matrix, roc_auc_score, f1_score, brier_score_loss

In [86]:
rng = np.random.RandomState(seed=12345)
idx = np.arange(len(y_outcome))

y_pred = np.asarray(y_pred)
y_pred_binary = np.asarray(y_pred_binary)
y = np.asarray(y_outcome)
X_dropped = np.asarray(X_features_imputed)

test_roc_auc = []
test_f1 = []
test_ftwos = []
test_brier = []
test_false_neg = []
test_false_pos = []

for i in range(200): # bootstrap with 200 rounds: random sampling with replacement of the predictions

    pred_idx = rng.choice(idx, size=len(idx), replace=True)
    
    roc_auc_test_boot = roc_auc_score(y_score=y_pred[pred_idx], y_true=y[pred_idx])
    f1_test_boot = f1_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx])
    f2_test_boot = fbeta_score(y_pred=y_pred_binary[pred_idx], y_true=y[pred_idx], beta=2)
    brier_test_boot = brier_score_loss(y_proba=y_pred[pred_idx], y_true=y[pred_idx])
    false_neg_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[1,0]
    false_pos_test_boot = confusion_matrix(y[pred_idx], y_pred_binary[pred_idx])[0,1]
    
    test_roc_auc.append(roc_auc_test_boot)
    test_f1.append(f1_test_boot)
    test_ftwos.append(f2_test_boot)
    test_brier.append(brier_test_boot)
    test_false_neg.append(false_neg_test_boot/len(idx)*100)
    test_false_pos.append(false_pos_test_boot/len(idx)*100)


In [87]:
print("Classification performance\n")

bootstrap_roc_auc_test_mean = np.mean(test_roc_auc)
ci_lower = np.percentile(test_roc_auc, 2.5)     # 2.5 percentile (alpha=0.025)
ci_upper = np.percentile(test_roc_auc, 97.5)
print(f"ROC AUC:         {bootstrap_roc_auc_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f1_test_mean = np.mean(test_f1)
ci_lower = np.percentile(test_f1, 2.5)
ci_upper = np.percentile(test_f1, 97.5)
print(f"F1:              {bootstrap_f1_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_f2_test_mean = np.mean(test_ftwos)
ci_lower = np.percentile(test_ftwos, 2.5)
ci_upper = np.percentile(test_ftwos, 97.5)
print(f"F2:              {bootstrap_f2_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_brier_test_mean = np.mean(test_brier)
ci_lower = np.percentile(test_brier, 2.5)
ci_upper = np.percentile(test_brier, 97.5)
print(f"Brier loss:      {bootstrap_brier_test_mean:.2f};   95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_neg_test_mean = np.mean(test_false_neg)
ci_lower = np.percentile(test_false_neg, 2.5)
ci_upper = np.percentile(test_false_neg, 97.5)
print(f"False negatives: {bootstrap_false_neg_test_mean:.2f}%;  95% CI {ci_lower:.2f}-{ci_upper:.2f}")

bootstrap_false_pos_test_mean = np.mean(test_false_pos)
ci_lower = np.percentile(test_false_pos, 2.5)
ci_upper = np.percentile(test_false_pos, 97.5)
print(f"False negatives: {bootstrap_false_pos_test_mean:.2f}%; 95% CI {ci_lower:.2f}-{ci_upper:.2f}")

Classification performance

ROC AUC:         0.88;   95% CI 0.81-0.94
F1:              0.55;   95% CI 0.42-0.67
F2:              0.61;   95% CI 0.47-0.74
Brier loss:      0.06;   95% CI 0.04-0.08
False negatives: 2.06%;  95% CI 1.12-3.19
False negatives: 4.36%; 95% CI 2.43-6.37
